In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW40NT-MakeConcordance-Matthew.py
"""
Created on Thu Jul 14 22:43:17 2016

@author: keith
"""

from BibleWordList2 import *

Matthew01 = [
   ['Matthew','Matt','01','001','1',"The book of the generation of Jesus Christ, the son of David, the son of Abraham."],
   ['Matthew','Matt','01','002','2',"Abraham begat Isaac; and Isaac begat Jacob; and Jacob begat Judas and his brethren;"],
['Matthew','Matt','01','003','3',"And Judas begat Phares and Zara of Thamar; and Phares begat Esrom; and Esrom begat Aram;"],
['Matthew','Matt','01','004','4',"And Aram begat Aminadab; and Aminadab begat Naasson; and Naasson begat Salmon;"],
['Matthew','Matt','01','005','5',"And Salmon begat Booz of Rachab; and Booz begat Obed of Ruth; and Obed begat Jesse;"],
['Matthew','Matt','01','006','6',"And Jesse begat David the king; and David the king begat Solomon of her \emph{that} \emph{had} \emph{been} \emph{the} \emph{wife} of Urias;"],
['Matthew','Matt','01','007','7',"And Solomon begat Roboam; and Roboam begat Abia; and Abia begat Asa;"],
['Matthew','Matt','01','008','8',"And Asa begat Josaphat; and Josaphat begat Joram; and Joram begat Ozias;"],
['Matthew','Matt','01','009','9',"And Ozias begat Joatham; and Joatham begat Achaz; and Achaz begat Ezekias;"],
['Matthew','Matt','01','010','10',"And Ezekias begat Manasses; and Manasses begat Amon; and Amon begat Josias;"],
['Matthew','Matt','01','011','11',"And Josias begat Jechonias and his brethren, about the time they were carried away to Babylon:"],
['Matthew','Matt','01','012','12',"And after they were brought to Babylon, Jechonias begat Salathiel; and Salathiel begat Zorobabel;"],
['Matthew','Matt','01','013','13',"And Zorobabel begat Abiud; and Abiud begat Eliakim; and Eliakim begat Azor;"],
['Matthew','Matt','01','014','14',"And Azor begat Sadoc; and Sadoc begat Achim; and Achim begat Eliud;"],
['Matthew','Matt','01','015','15',"And Eliud begat Eleazar; and Eleazar begat Matthan; and Matthan begat Jacob;"],
['Matthew','Matt','01','016','16',"And Jacob begat Joseph the husband of Mary, of whom was born Jesus, who is called Christ."],
['Matthew','Matt','01','017','17',"So all the generations from Abraham to David \emph{are} fourteen generations; and from David until the carrying away into Babylon \emph{are} fourteen generations; and from the carrying away into Babylon unto Christ \emph{are} fourteen generations."],
['Matthew','Matt','01','018','18',"Now the birth of Jesus Christ was on this wise: When as his mother Mary was espoused to Joseph, before they came together, she was found with child of the Holy Ghost."],
['Matthew','Matt','01','019','19',"Then Joseph her husband, being a just \emph{man}, and not willing to make her a publick example, was minded to put her away privily."],
['Matthew','Matt','01','020','20',"But while he thought on these things, behold, the angel of the Lord appeared unto him in a dream, saying, Joseph, thou son of David, fear not to take unto thee Mary thy wife: for that which is conceived in her is of the Holy Ghost."],
['Matthew','Matt','01','021','21',"And she shall bring forth a son, and thou shalt call his name JESUS: for he shall save his people from their sins."],
['Matthew','Matt','01','022','22',"Now all this was done, that it might be fulfilled which was spoken of the Lord by the prophet, saying,"],
['Matthew','Matt','01','023','23',"Behold, a virgin shall be with child, and shall bring forth a son, and they shall call his name Emmanuel, which being interpreted is, God with us."],
['Matthew','Matt','01','024','24',"Then Joseph being raised from sleep did as the angel of the Lord had bidden him, and took unto him his wife:"],
['Matthew','Matt','01','025','25',"And knew her not till she had brought forth her firstborn son: and he called his name JESUS."]
]

Matthew02 = [
    ['Matthew','Matt','02','001','1',"Now when Jesus was born in Bethlehem of Judaea in the days of Herod the king, behold, there came wise men from the east to Jerusalem,"],
['Matthew','Matt','02','002','2',"Saying, Where is he that is born King of the Jews? for we have seen his star in the east, and are come to worship him."],
['Matthew','Matt','02','003','3',"When Herod the king had heard \emph{these} \emph{things}, he was troubled, and all Jerusalem with him."],
['Matthew','Matt','02','004','4',"And when he had gathered all the chief priests and scribes of the people together, he demanded of them where Christ should be born."],
['Matthew','Matt','02','005','5',"And they said unto him, In Bethlehem of Judaea: for thus it is written by the prophet,"],
['Matthew','Matt','02','006','6',"And thou Bethlehem, \emph{in} the land of Juda, art not the least among the princes of Juda: for out of thee shall come a Governor, that shall rule my people Israel."],
['Matthew','Matt','02','007','7',"Then Herod, when he had privily called the wise men, enquired of them diligently what time the star appeared."],
['Matthew','Matt','02','008','8',"And he sent them to Bethlehem, and said, Go and search diligently for the young child; and when ye have found \emph{him}, bring me word again, that I may come and worship him also."],
['Matthew','Matt','02','009','9',"When they had heard the king, they departed; and, lo, the star, which they saw in the east, went before them, till it came and stood over where the young child was."],
['Matthew','Matt','02','010','10',"When they saw the star, they rejoiced with exceeding great joy."],
['Matthew','Matt','02','011','11',"And when they were come into the house, they saw the young child with Mary his mother, and fell down, and worshipped him: and when they had opened their treasures, they presented unto him gifts; gold, and frankincense, and myrrh."],
['Matthew','Matt','02','012','12',"And being warned of God in a dream that they should not return to Herod, they departed into their own country another way."],
['Matthew','Matt','02','013','13',"And when they were departed, behold, the angel of the Lord appeareth to Joseph in a dream, saying, Arise, and take the young child and his mother, and flee into Egypt, and be thou there until I bring thee word: for Herod will seek the young child to destroy him."],
['Matthew','Matt','02','014','14',"When he arose, he took the young child and his mother by night, and departed into Egypt:"],
['Matthew','Matt','02','015','15',"And was there until the death of Herod: that it might be fulfilled which was spoken of the Lord by the prophet, saying, Out of Egypt have I called my son."],
['Matthew','Matt','02','016','16',"Then Herod, when he saw that he was mocked of the wise men, was exceeding wroth, and sent forth, and slew all the children that were in Bethlehem, and in all the coasts thereof, from two years old and under, according to the time which he had diligently enquired of the wise men."],
['Matthew','Matt','02','017','17',"Then was fulfilled that which was spoken by Jeremy the prophet, saying,"],
['Matthew','Matt','02','018','18',"In Rama was there a voice heard, lamentation, and weeping, and great mourning, Rachel weeping \emph{for} her children, and would not be comforted, because they are not."],
['Matthew','Matt','02','019','19',"But when Herod was dead, behold, an angel of the Lord appeareth in a dream to Joseph in Egypt,"],
['Matthew','Matt','02','020','20',"Saying, Arise, and take the young child and his mother, and go into the land of Israel: for they are dead which sought the young child's life."],
['Matthew','Matt','02','021','21',"And he arose, and took the young child and his mother, and came into the land of Israel."],
['Matthew','Matt','02','022','22',"But when he heard that Archelaus did reign in Judaea in the room of his father Herod, he was afraid to go thither: notwithstanding, being warned of God in a dream, he turned aside into the parts of Galilee:"],
['Matthew','Matt','02','023','23',"And he came and dwelt in a city called Nazareth: that it might be fulfilled which was spoken by the prophets, He shall be called a Nazarene."]
]

Matthew03 = [
    ['Matthew','Matt','03','001','1',"In those days came John the Baptist, preaching in the wilderness of Judaea,"],
['Matthew','Matt','03','002','2',"And saying, Repent ye: for the kingdom of heaven is at hand."],
['Matthew','Matt','03','003','3',"For this is he that was spoken of by the prophet Esaias, saying, The voice of one crying in the wilderness, Prepare ye the way of the Lord, make his paths straight."],
['Matthew','Matt','03','004','4',"And the same John had his raiment of camel's hair, and a leathern girdle about his loins; and his meat was locusts and wild honey."],
['Matthew','Matt','03','005','5',"Then went out to him Jerusalem, and all Judaea, and all the region round about Jordan,"],
['Matthew','Matt','03','006','6',"And were baptized of him in Jordan, confessing their sins."],
['Matthew','Matt','03','007','7',"But when he saw many of the Pharisees and Sadducees come to his baptism, he said unto them, O generation of vipers, who hath warned you to flee from the wrath to come?"],
['Matthew','Matt','03','008','8',"Bring forth therefore fruits meet for repentance:"],
['Matthew','Matt','03','009','9',"And think not to say within yourselves, We have Abraham to \emph{our} father: for I say unto you, that God is able of these stones to raise up children unto Abraham."],
['Matthew','Matt','03','010','10',"And now also the axe is laid unto the root of the trees: therefore every tree which bringeth not forth good fruit is hewn down, and cast into the fire."],
['Matthew','Matt','03','011','11',"I indeed baptize you with water unto repentance: but he that cometh after me is mightier than I, whose shoes I am not worthy to bear: he shall baptize you with the Holy Ghost, and \emph{with} fire:"],
['Matthew','Matt','03','012','12',"Whose fan \emph{is} in his hand, and he will throughly purge his floor, and gather his wheat into the garner; but he will burn up the chaff with unquenchable fire."],
['Matthew','Matt','03','013','13',"Then cometh Jesus from Galilee to Jordan unto John, to be baptized of him."],
['Matthew','Matt','03','014','14',"But John forbad him, saying, I have need to be baptized of thee, and comest thou to me?"],
['Matthew','Matt','03','015','15',"And Jesus answering said unto him, Suffer \emph{it} \emph{to} \emph{be} \emph{so} now: for thus it becometh us to fulfil all righteousness. Then he suffered him."],
['Matthew','Matt','03','016','16',"And Jesus, when he was baptized, went up straightway out of the water: and, lo, the heavens were opened unto him, and he saw the Spirit of God descending like a dove, and lighting upon him:"],
['Matthew','Matt','03','017','17',"And lo a voice from heaven, saying, This is my beloved Son, in whom I am well pleased."]
]

Matthew04 = [
    ['Matthew','Matt','04','001','1',"Then was Jesus led up of the Spirit into the wilderness to be tempted of the devil."],
['Matthew','Matt','04','002','2',"And when he had fasted forty days and forty nights, he was afterward an hungred."],
['Matthew','Matt','04','003','3',"And when the tempter came to him, he said, If thou be the Son of God, command that these stones be made bread."],
['Matthew','Matt','04','004','4',"But he answered and said, It is written, Man shall not live by bread alone, but by every word that proceedeth out of the mouth of God."],
['Matthew','Matt','04','005','5',"Then the devil taketh him up into the holy city, and setteth him on a pinnacle of the temple,"],
['Matthew','Matt','04','006','6',"And saith unto him, If thou be the Son of God, cast thyself down: for it is written, He shall give his angels charge concerning thee: and in \emph{their} hands they shall bear thee up, lest at any time thou dash thy foot against a stone."],
['Matthew','Matt','04','007','7',"Jesus said unto him, It is written again, Thou shalt not tempt the Lord thy God."],
['Matthew','Matt','04','008','8',"Again, the devil taketh him up into an exceeding high mountain, and sheweth him all the kingdoms of the world, and the glory of them;"],
['Matthew','Matt','04','009','9',"And saith unto him, All these things will I give thee, if thou wilt fall down and worship me."],
['Matthew','Matt','04','010','10',"Then saith Jesus unto him, Get thee hence, Satan: for it is written, Thou shalt worship the Lord thy God, and him only shalt thou serve."],
['Matthew','Matt','04','011','11',"Then the devil leaveth him, and, behold, angels came and ministered unto him."],
['Matthew','Matt','04','012','12',"Now when Jesus had heard that John was cast into prison, he departed into Galilee;"],
['Matthew','Matt','04','013','13',"And leaving Nazareth, he came and dwelt in Capernaum, which is upon the sea coast, in the borders of Zabulon and Nephthalim:"],
['Matthew','Matt','04','014','14',"That it might be fulfilled which was spoken by Esaias the prophet, saying,"],
['Matthew','Matt','04','015','15',"The land of Zabulon, and the land of Nephthalim, \emph{by} the way of the sea, beyond Jordan, Galilee of the Gentiles;"],
['Matthew','Matt','04','016','16',"The people which sat in darkness saw great light; and to them which sat in the region and shadow of death light is sprung up."],
['Matthew','Matt','04','017','17',"From that time Jesus began to preach, and to say, Repent: for the kingdom of heaven is at hand."],
['Matthew','Matt','04','018','18',"And Jesus, walking by the sea of Galilee, saw two brethren, Simon called Peter, and Andrew his brother, casting a net into the sea: for they were fishers."],
['Matthew','Matt','04','019','19',"And he saith unto them, Follow me, and I will make you fishers of men."],
['Matthew','Matt','04','020','20',"And they straightway left \emph{their} nets, and followed him."],
['Matthew','Matt','04','021','21',"And going on from thence, he saw other two brethren, James \emph{the} \emph{son} of Zebedee, and John his brother, in a ship with Zebedee their father, mending their nets; and he called them."],
['Matthew','Matt','04','022','22',"And they immediately left the ship and their father, and followed him."],
['Matthew','Matt','04','023','23',"And Jesus went about all Galilee, teaching in their synagogues, and preaching the gospel of the kingdom, and healing all manner of sickness and all manner of disease among the people."],
['Matthew','Matt','04','024','24',"And his fame went throughout all Syria: and they brought unto him all sick people that were taken with divers diseases and torments, and those which were possessed with devils, and those which were lunatick, and those that had the palsy; and he healed them."],
['Matthew','Matt','04','025','25',"And there followed him great multitudes of people from Galilee, and \emph{from} Decapolis, and \emph{from} Jerusalem, and \emph{from} Judaea, and \emph{from} beyond Jordan."]
]

Matthew05 = [
    ['Matthew','Matt','05','001','1',"And seeing the multitudes, he went up into a mountain: and when he was set, his disciples came unto him:"],
['Matthew','Matt','05','002','2',"And he opened his mouth, and taught them, saying,"],
['Matthew','Matt','05','003','3',"Blessed \emph{are} the poor in spirit: for their's is the kingdom of heaven."],
['Matthew','Matt','05','004','4',"Blessed \emph{are} they that mourn: for they shall be comforted."],
['Matthew','Matt','05','005','5',"Blessed \emph{are} the meek: for they shall inherit the earth."],
['Matthew','Matt','05','006','6',"Blessed \emph{are} they which do hunger and thirst after righteousness: for they shall be filled."],
['Matthew','Matt','05','007','7',"Blessed \emph{are} the merciful: for they shall obtain mercy."],
['Matthew','Matt','05','008','8',"Blessed \emph{are} the pure in heart: for they shall see God."],
['Matthew','Matt','05','009','9',"Blessed \emph{are} the peacemakers: for they shall be called the children of God."],
['Matthew','Matt','05','010','10',"Blessed \emph{are} they which are persecuted for righteousness' sake: for their's is the kingdom of heaven."],
['Matthew','Matt','05','011','11',"Blessed are ye, when \emph{men} shall revile you, and persecute \emph{you}, and shall say all manner of evil against you falsely, for my sake."],
['Matthew','Matt','05','012','12',"Rejoice, and be exceeding glad: for great \emph{is} your reward in heaven: for so persecuted they the prophets which were before you."],
['Matthew','Matt','05','013','13',"Ye are the salt of the earth: but if the salt have lost his savour, wherewith shall it be salted? it is thenceforth good for nothing, but to be cast out, and to be trodden under foot of men."],
['Matthew','Matt','05','014','14',"Ye are the light of the world. A city that is set on an hill cannot be hid."],
['Matthew','Matt','05','015','15',"Neither do men light a candle, and put it under a bushel, but on a candlestick; and it giveth light unto all that are in the house."],
['Matthew','Matt','05','016','16',"Let your light so shine before men, that they may see your good works, and glorify your Father which is in heaven."],
['Matthew','Matt','05','017','17',"Think not that I am come to destroy the law, or the prophets: I am not come to destroy, but to fulfil."],
['Matthew','Matt','05','018','18',"For verily I say unto you, Till heaven and earth pass, one jot or one tittle shall in no wise pass from the law, till all be fulfilled."],
['Matthew','Matt','05','019','19',"Whosoever therefore shall break one of these least commandments, and shall teach men so, he shall be called the least in the kingdom of heaven: but whosoever shall do and teach \emph{them}, the same shall be called great in the kingdom of heaven."],
['Matthew','Matt','05','020','20',"For I say unto you, That except your righteousness shall exceed \emph{the} \emph{righteousness} of the scribes and Pharisees, ye shall in no case enter into the kingdom of heaven."],
['Matthew','Matt','05','021','21',"Ye have heard that it was said by them of old time, Thou shalt not kill; and whosoever shall kill shall be in danger of the judgment:"],
['Matthew','Matt','05','022','22',"But I say unto you, That whosoever is angry with his brother without a cause shall be in danger of the judgment: and whosoever shall say to his brother, Raca, shall be in danger of the council: but whosoever shall say, Thou fool, shall be in danger of hell fire."],
['Matthew','Matt','05','023','23',"Therefore if thou bring thy gift to the altar, and there rememberest that thy brother hath ought against thee;"],
['Matthew','Matt','05','024','24',"Leave there thy gift before the altar, and go thy way; first be reconciled to thy brother, and then come and offer thy gift."],
['Matthew','Matt','05','025','25',"Agree with thine adversary quickly, whiles thou art in the way with him; lest at any time the adversary deliver thee to the judge, and the judge deliver thee to the officer, and thou be cast into prison."],
['Matthew','Matt','05','026','26',"Verily I say unto thee, Thou shalt by no means come out thence, till thou hast paid the uttermost farthing."],
['Matthew','Matt','05','027','27',"Ye have heard that it was said by them of old time, Thou shalt not commit adultery:"],
['Matthew','Matt','05','028','28',"But I say unto you, That whosoever looketh on a woman to lust after her hath committed adultery with her already in his heart."],
['Matthew','Matt','05','029','29',"And if thy right eye offend thee, pluck it out, and cast \emph{it} from thee: for it is profitable for thee that one of thy members should perish, and not \emph{that} thy whole body should be cast into hell."],
['Matthew','Matt','05','030','30',"And if thy right hand offend thee, cut it off, and cast \emph{it} from thee: for it is profitable for thee that one of thy members should perish, and not \emph{that} thy whole body should be cast into hell."],
['Matthew','Matt','05','031','31',"It hath been said, Whosoever shall put away his wife, let him give her a writing of divorcement:"],
['Matthew','Matt','05','032','32',"But I say unto you, That whosoever shall put away his wife, saving for the cause of fornication, causeth her to commit adultery: and whosoever shall marry her that is divorced committeth adultery."],
['Matthew','Matt','05','033','33',"Again, ye have heard that it hath been said by them of old time, Thou shalt not forswear thyself, but shalt perform unto the Lord thine oaths:"],
['Matthew','Matt','05','034','34',"But I say unto you, Swear not at all; neither by heaven; for it is God's throne:"],
['Matthew','Matt','05','035','35',"Nor by the earth; for it is his footstool: neither by Jerusalem; for it is the city of the great King."],
['Matthew','Matt','05','036','36',"Neither shalt thou swear by thy head, because thou canst not make one hair white or black."],
['Matthew','Matt','05','037','37',"But let your communication be, Yea, yea; Nay, nay: for whatsoever is more than these cometh of evil."],
['Matthew','Matt','05','038','38',"Ye have heard that it hath been said, An eye for an eye, and a tooth for a tooth:"],
['Matthew','Matt','05','039','39',"But I say unto you, That ye resist not evil: but whosoever shall smite thee on thy right cheek, turn to him the other also."],
['Matthew','Matt','05','040','40',"And if any man will sue thee at the law, and take away thy coat, let him have \emph{thy} cloke also."],
['Matthew','Matt','05','041','41',"And whosoever shall compel thee to go a mile, go with him twain."],
['Matthew','Matt','05','042','42',"Give to him that asketh thee, and from him that would borrow of thee turn not thou away."],
['Matthew','Matt','05','043','43',"Ye have heard that it hath been said, Thou shalt love thy neighbour, and hate thine enemy."],
['Matthew','Matt','05','044','44',"But I say unto you, Love your enemies, bless them that curse you, do good to them that hate you, and pray for them which despitefully use you, and persecute you;"],
['Matthew','Matt','05','045','45',"That ye may be the children of your Father which is in heaven: for he maketh his sun to rise on the evil and on the good, and sendeth rain on the just and on the unjust."],
['Matthew','Matt','05','046','46',"For if ye love them which love you, what reward have ye? do not even the publicans the same?"],
['Matthew','Matt','05','047','47',"And if ye salute your brethren only, what do ye more \emph{than} \emph{others}? do not even the publicans so?"],
['Matthew','Matt','05','048','48',"Be ye therefore perfect, even as your Father which is in heaven is perfect."]
]

Matthew06 = [
    ['Matthew','Matt','06','001','1',"Take heed that ye do not your alms before men, to be seen of them: otherwise ye have no reward of your Father which is in heaven."],
['Matthew','Matt','06','002','2',"Therefore when thou doest \emph{thine} alms, do not sound a trumpet before thee, as the hypocrites do in the synagogues and in the streets, that they may have glory of men. Verily I say unto you, They have their reward."],
['Matthew','Matt','06','003','3',"But when thou doest alms, let not thy left hand know what thy right hand doeth:"],
['Matthew','Matt','06','004','4',"That thine alms may be in secret: and thy Father which seeth in secret himself shall reward thee openly."],
['Matthew','Matt','06','005','5',"And when thou prayest, thou shalt not be as the hypocrites \emph{are}: for they love to pray standing in the synagogues and in the corners of the streets, that they may be seen of men. Verily I say unto you, They have their reward."],
['Matthew','Matt','06','006','6',"But thou, when thou prayest, enter into thy closet, and when thou hast shut thy door, pray to thy Father which is in secret; and thy Father which seeth in secret shall reward thee openly."],
['Matthew','Matt','06','007','7',"But when ye pray, use not vain repetitions, as the heathen \emph{do}: for they think that they shall be heard for their much speaking."],
['Matthew','Matt','06','008','8',"Be not ye therefore like unto them: for your Father knoweth what things ye have need of, before ye ask him."],
['Matthew','Matt','06','009','9',"After this manner therefore pray ye: Our Father which art in heaven, Hallowed be thy name."],
['Matthew','Matt','06','010','10',"Thy kingdom come. Thy will be done in earth, as \emph{it} \emph{is} in heaven."],
['Matthew','Matt','06','011','11',"Give us this day our daily bread."],
['Matthew','Matt','06','012','12',"And forgive us our debts, as we forgive our debtors."],
['Matthew','Matt','06','013','13',"And lead us not into temptation, but deliver us from evil: For thine is the kingdom, and the power, and the glory, for ever. Amen."],
['Matthew','Matt','06','014','14',"For if ye forgive men their trespasses, your heavenly Father will also forgive you:"],
['Matthew','Matt','06','015','15',"But if ye forgive not men their trespasses, neither will your Father forgive your trespasses."],
['Matthew','Matt','06','016','16',"Moreover when ye fast, be not, as the hypocrites, of a sad countenance: for they disfigure their faces, that they may appear unto men to fast. Verily I say unto you, They have their reward."],
['Matthew','Matt','06','017','17',"But thou, when thou fastest, anoint thine head, and wash thy face;"],
['Matthew','Matt','06','018','18',"That thou appear not unto men to fast, but unto thy Father which is in secret: and thy Father, which seeth in secret, shall reward thee openly."],
['Matthew','Matt','06','019','19',"Lay not up for yourselves treasures upon earth, where moth and rust doth corrupt, and where thieves break through and steal:"],
['Matthew','Matt','06','020','20',"But lay up for yourselves treasures in heaven, where neither moth nor rust doth corrupt, and where thieves do not break through nor steal:"],
['Matthew','Matt','06','021','21',"For where your treasure is, there will your heart be also."],
['Matthew','Matt','06','022','22',"The light of the body is the eye: if therefore thine eye be single, thy whole body shall be full of light."],
['Matthew','Matt','06','023','23',"But if thine eye be evil, thy whole body shall be full of darkness. If therefore the light that is in thee be darkness, how great \emph{is} that darkness!"],
['Matthew','Matt','06','024','24',"No man can serve two masters: for either he will hate the one, and love the other; or else he will hold to the one, and despise the other. Ye cannot serve God and mammon."],
['Matthew','Matt','06','025','25',"Therefore I say unto you, Take no thought for your life, what ye shall eat, or what ye shall drink; nor yet for your body, what ye shall put on. Is not the life more than meat, and the body than raiment?"],
['Matthew','Matt','06','026','26',"Behold the fowls of the air: for they sow not, neither do they reap, nor gather into barns; yet your heavenly Father feedeth them. Are ye not much better than they?"],
['Matthew','Matt','06','027','27',"Which of you by taking thought can add one cubit unto his stature?"],
['Matthew','Matt','06','028','28',"And why take ye thought for raiment? Consider the lilies of the field, how they grow; they toil not, neither do they spin:"],
['Matthew','Matt','06','029','29',"And yet I say unto you, That even Solomon in all his glory was not arrayed like one of these."],
['Matthew','Matt','06','030','30',"Wherefore, if God so clothe the grass of the field, which to day is, and to morrow is cast into the oven, \emph{shall} \emph{he} not much more \emph{clothe} you, O ye of little faith?"],
['Matthew','Matt','06','031','31',"Therefore take no thought, saying, What shall we eat? or, What shall we drink? or, Wherewithal shall we be clothed?"],
['Matthew','Matt','06','032','32',"(For after all these things do the Gentiles seek:) for your heavenly Father knoweth that ye have need of all these things."],
['Matthew','Matt','06','033','33',"But seek ye first the kingdom of God, and his righteousness; and all these things shall be added unto you."],
['Matthew','Matt','06','034','34',"Take therefore no thought for the morrow: for the morrow shall take thought for the things of itself. Sufficient unto the day \emph{is} the evil thereof."]
]

Matthew07 = [
    ['Matthew','Matt','07','001','1',"Judge not, that ye be not judged."],
['Matthew','Matt','07','002','2',"For with what judgment ye judge, ye shall be judged: and with what measure ye mete, it shall be measured to you again."],
['Matthew','Matt','07','003','3',"And why beholdest thou the mote that is in thy brother's eye, but considerest not the beam that is in thine own eye?"],
['Matthew','Matt','07','004','4',"Or how wilt thou say to thy brother, Let me pull out the mote out of thine eye; and, behold, a beam \emph{is} in thine own eye?"],
['Matthew','Matt','07','005','5',"Thou hypocrite, first cast out the beam out of thine own eye; and then shalt thou see clearly to cast out the mote out of thy brother's eye."],
['Matthew','Matt','07','006','6',"Give not that which is holy unto the dogs, neither cast ye your pearls before swine, lest they trample them under their feet, and turn again and rend you."],
['Matthew','Matt','07','007','7',"Ask, and it shall be given you; seek, and ye shall find; knock, and it shall be opened unto you:"],
['Matthew','Matt','07','008','8',"For every one that asketh receiveth; and he that seeketh findeth; and to him that knocketh it shall be opened."],
['Matthew','Matt','07','009','9',"Or what man is there of you, whom if his son ask bread, will he give him a stone?"],
['Matthew','Matt','07','010','10',"Or if he ask a fish, will he give him a serpent?"],
['Matthew','Matt','07','011','11',"If ye then, being evil, know how to give good gifts unto your children, how much more shall your Father which is in heaven give good things to them that ask him?"],
['Matthew','Matt','07','012','12',"Therefore all things whatsoever ye would that men should do to you, do ye even so to them: for this is the law and the prophets."],
['Matthew','Matt','07','013','13',"Enter ye in at the strait gate: for wide \emph{is} the gate, and broad \emph{is} the way, that leadeth to destruction, and many there be which go in thereat:"],
['Matthew','Matt','07','014','14',"Because strait \emph{is} the gate, and narrow \emph{is} the way, which leadeth unto life, and few there be that find it."],
['Matthew','Matt','07','015','15',"Beware of false prophets, which come to you in sheep's clothing, but inwardly they are ravening wolves."],
['Matthew','Matt','07','016','16',"Ye shall know them by their fruits. Do men gather grapes of thorns, or figs of thistles?"],
['Matthew','Matt','07','017','17',"Even so every good tree bringeth forth good fruit; but a corrupt tree bringeth forth evil fruit."],
['Matthew','Matt','07','018','18',"A good tree cannot bring forth evil fruit, neither \emph{can} a corrupt tree bring forth good fruit."],
['Matthew','Matt','07','019','19',"Every tree that bringeth not forth good fruit is hewn down, and cast into the fire."],
['Matthew','Matt','07','020','20',"Wherefore by their fruits ye shall know them."],
['Matthew','Matt','07','021','21',"Not every one that saith unto me, Lord, Lord, shall enter into the kingdom of heaven; but he that doeth the will of my Father which is in heaven."],
['Matthew','Matt','07','022','22',"Many will say to me in that day, Lord, Lord, have we not prophesied in thy name? and in thy name have cast out devils? and in thy name done many wonderful works?"],
['Matthew','Matt','07','023','23',"And then will I profess unto them, I never knew you: depart from me, ye that work iniquity."],
['Matthew','Matt','07','024','24',"Therefore whosoever heareth these sayings of mine, and doeth them, I will liken him unto a wise man, which built his house upon a rock:"],
['Matthew','Matt','07','025','25',"And the rain descended, and the floods came, and the winds blew, and beat upon that house; and it fell not: for it was founded upon a rock."],
['Matthew','Matt','07','026','26',"And every one that heareth these sayings of mine, and doeth them not, shall be likened unto a foolish man, which built his house upon the sand:"],
['Matthew','Matt','07','027','27',"And the rain descended, and the floods came, and the winds blew, and beat upon that house; and it fell: and great was the fall of it."],
['Matthew','Matt','07','028','28',"And it came to pass, when Jesus had ended these sayings, the people were astonished at his doctrine:"],
['Matthew','Matt','07','029','29',"For he taught them as \emph{one} having authority, and not as the scribes."]
]

Matthew08 = [['Matthew','Matt','08','001','1',"When he was come down from the mountain, great multitudes followed him."],
['Matthew','Matt','08','002','2',"And, behold, there came a leper and worshipped him, saying, Lord, if thou wilt, thou canst make me clean."],
['Matthew','Matt','08','003','3',"And Jesus put forth \emph{his} hand, and touched him, saying, I will; be thou clean. And immediately his leprosy was cleansed."],
['Matthew','Matt','08','004','4',"And Jesus saith unto him, See thou tell no man; but go thy way, shew thyself to the priest, and offer the gift that Moses commanded, for a testimony unto them."],
['Matthew','Matt','08','005','5',"And when Jesus was entered into Capernaum, there came unto him a centurion, beseeching him,"],
['Matthew','Matt','08','006','6',"And saying, Lord, my servant lieth at home sick of the palsy, grievously tormented."],
['Matthew','Matt','08','007','7',"And Jesus saith unto him, I will come and heal him."],
['Matthew','Matt','08','008','8',"The centurion answered and said, Lord, I am not worthy that thou shouldest come under my roof: but speak the word only, and my servant shall be healed."],
['Matthew','Matt','08','009','9',"For I am a man under authority, having soldiers under me: and I say to this \emph{man}, Go, and he goeth; and to another, Come, and he cometh; and to my servant, Do this, and he doeth \emph{it}."],
['Matthew','Matt','08','010','10',"When Jesus heard \emph{it}, he marvelled, and said to them that followed, Verily I say unto you, I have not found so great faith, no, not in Israel."],
['Matthew','Matt','08','011','11',"And I say unto you, That many shall come from the east and west, and shall sit down with Abraham, and Isaac, and Jacob, in the kingdom of heaven."],
['Matthew','Matt','08','012','12',"But the children of the kingdom shall be cast out into outer darkness: there shall be weeping and gnashing of teeth."],
['Matthew','Matt','08','013','13',"And Jesus said unto the centurion, Go thy way; and as thou hast believed, \emph{so} be it done unto thee. And his servant was healed in the selfsame hour."],
['Matthew','Matt','08','014','14',"And when Jesus was come into Peter's house, he saw his wife's mother laid, and sick of a fever."],
['Matthew','Matt','08','015','15',"And he touched her hand, and the fever left her: and she arose, and ministered unto them."],
['Matthew','Matt','08','016','16',"When the even was come, they brought unto him many that were possessed with devils: and he cast out the spirits with \emph{his} word, and healed all that were sick:"],
['Matthew','Matt','08','017','17',"That it might be fulfilled which was spoken by Esaias the prophet, saying, Himself took our infirmities, and bare \emph{our} sicknesses."],
['Matthew','Matt','08','018','18',"Now when Jesus saw great multitudes about him, he gave commandment to depart unto the other side."],
['Matthew','Matt','08','019','19',"And a certain scribe came, and said unto him, Master, I will follow thee whithersoever thou goest."],
['Matthew','Matt','08','020','20',"And Jesus saith unto him, The foxes have holes, and the birds of the air \emph{have} nests; but the Son of man hath not where to lay \emph{his} head."],
['Matthew','Matt','08','021','21',"And another of his disciples said unto him, Lord, suffer me first to go and bury my father."],
['Matthew','Matt','08','022','22',"But Jesus said unto him, Follow me; and let the dead bury their dead."],
['Matthew','Matt','08','023','23',"And when he was entered into a ship, his disciples followed him."],
['Matthew','Matt','08','024','24',"And, behold, there arose a great tempest in the sea, insomuch that the ship was covered with the waves: but he was asleep."],
['Matthew','Matt','08','025','25',"And his disciples came to \emph{him}, and awoke him, saying, Lord, save us: we perish."],
['Matthew','Matt','08','026','26',"And he saith unto them, Why are ye fearful, O ye of little faith? Then he arose, and rebuked the winds and the sea; and there was a great calm."],
['Matthew','Matt','08','027','27',"But the men marvelled, saying, What manner of man is this, that even the winds and the sea obey him!"],
['Matthew','Matt','08','028','28',"And when he was come to the other side into the country of the Gergesenes, there met him two possessed with devils, coming out of the tombs, exceeding fierce, so that no man might pass by that way."],
['Matthew','Matt','08','029','29',"And, behold, they cried out, saying, What have we to do with thee, Jesus, thou Son of God? art thou come hither to torment us before the time?"],
['Matthew','Matt','08','030','30',"And there was a good way off from them an herd of many swine feeding."],
['Matthew','Matt','08','031','31',"So the devils besought him, saying, If thou cast us out, suffer us to go away into the herd of swine."],
['Matthew','Matt','08','032','32',"And he said unto them, Go. And when they were come out, they went into the herd of swine: and, behold, the whole herd of swine ran violently down a steep place into the sea, and perished in the waters."],
['Matthew','Matt','08','033','33',"And they that kept them fled, and went their ways into the city, and told every thing, and what was befallen to the possessed of the devils."],
['Matthew','Matt','08','034','34',"And, behold, the whole city came out to meet Jesus: and when they saw him, they besought \emph{him} that he would depart out of their coasts."]
]

Matthew09 = [
    ['Matthew','Matt','09','001','1',"And he entered into a ship, and passed over, and came into his own city."],
['Matthew','Matt','09','002','2',"And, behold, they brought to him a man sick of the palsy, lying on a bed: and Jesus seeing their faith said unto the sick of the palsy; Son, be of good cheer; thy sins be forgiven thee."],
['Matthew','Matt','09','003','3',"And, behold, certain of the scribes said within themselves, This \emph{man} blasphemeth."],
['Matthew','Matt','09','004','4',"And Jesus knowing their thoughts said, Wherefore think ye evil in your hearts?"],
['Matthew','Matt','09','005','5',"For whether is easier, to say, \emph{Thy} sins be forgiven thee; or to say, Arise, and walk?"],
['Matthew','Matt','09','006','6',"But that ye may know that the Son of man hath power on earth to forgive sins, (then saith he to the sick of the palsy,) Arise, take up thy bed, and go unto thine house."],
['Matthew','Matt','09','007','7',"And he arose, and departed to his house."],
['Matthew','Matt','09','008','8',"But when the multitudes saw \emph{it}, they marvelled, and glorified God, which had given such power unto men."],
['Matthew','Matt','09','009','9',"And as Jesus passed forth from thence, he saw a man, named Matthew, sitting at the receipt of custom: and he saith unto him, Follow me. And he arose, and followed him."],
['Matthew','Matt','09','010','10',"And it came to pass, as Jesus sat at meat in the house, behold, many publicans and sinners came and sat down with him and his disciples."],
['Matthew','Matt','09','011','11',"And when the Pharisees saw \emph{it}, they said unto his disciples, Why eateth your Master with publicans and sinners?"],
['Matthew','Matt','09','012','12',"But when Jesus heard \emph{that}, he said unto them, They that be whole need not a physician, but they that are sick."],
['Matthew','Matt','09','013','13',"But go ye and learn what \emph{that} meaneth, I will have mercy, and not sacrifice: for I am not come to call the righteous, but sinners to repentance."],
['Matthew','Matt','09','014','14',"Then came to him the disciples of John, saying, Why do we and the Pharisees fast oft, but thy disciples fast not?"],
['Matthew','Matt','09','015','15',"And Jesus said unto them, Can the children of the bridechamber mourn, as long as the bridegroom is with them? but the days will come, when the bridegroom shall be taken from them, and then shall they fast."],
['Matthew','Matt','09','016','16',"No man putteth a piece of new cloth unto an old garment, for that which is put in to fill it up taketh from the garment, and the rent is made worse."],
['Matthew','Matt','09','017','17',"Neither do men put new wine into old bottles: else the bottles break, and the wine runneth out, and the bottles perish: but they put new wine into new bottles, and both are preserved."],
['Matthew','Matt','09','018','18',"While he spake these things unto them, behold, there came a certain ruler, and worshipped him, saying, My daughter is even now dead: but come and lay thy hand upon her, and she shall live."],
['Matthew','Matt','09','019','19',"And Jesus arose, and followed him, and \emph{so} \emph{did} his disciples."],
['Matthew','Matt','09','020','20',"And, behold, a woman, which was diseased with an issue of blood twelve years, came behind \emph{him}, and touched the hem of his garment:"],
['Matthew','Matt','09','021','21',"For she said within herself, If I may but touch his garment, I shall be whole."],
['Matthew','Matt','09','022','22',"But Jesus turned him about, and when he saw her, he said, Daughter, be of good comfort; thy faith hath made thee whole. And the woman was made whole from that hour."],
['Matthew','Matt','09','023','23',"And when Jesus came into the ruler's house, and saw the minstrels and the people making a noise,"],
['Matthew','Matt','09','024','24',"He said unto them, Give place: for the maid is not dead, but sleepeth. And they laughed him to scorn."],
['Matthew','Matt','09','025','25',"But when the people were put forth, he went in, and took her by the hand, and the maid arose."],
['Matthew','Matt','09','026','26',"And the fame hereof went abroad into all that land."],
['Matthew','Matt','09','027','27',"And when Jesus departed thence, two blind men followed him, crying, and saying, \emph{Thou} Son of David, have mercy on us."],
['Matthew','Matt','09','028','28',"And when he was come into the house, the blind men came to him: and Jesus saith unto them, Believe ye that I am able to do this? They said unto him, Yea, Lord."],
['Matthew','Matt','09','029','29',"Then touched he their eyes, saying, According to your faith be it unto you."],
['Matthew','Matt','09','030','30',"And their eyes were opened; and Jesus straitly charged them, saying, See \emph{that} no man know \emph{it}."],
['Matthew','Matt','09','031','31',"But they, when they were departed, spread abroad his fame in all that country."],
['Matthew','Matt','09','032','32',"As they went out, behold, they brought to him a dumb man possessed with a devil."],
['Matthew','Matt','09','033','33',"And when the devil was cast out, the dumb spake: and the multitudes marvelled, saying, It was never so seen in Israel."],
['Matthew','Matt','09','034','34',"But the Pharisees said, He casteth out devils through the prince of the devils."],
['Matthew','Matt','09','035','35',"And Jesus went about all the cities and villages, teaching in their synagogues, and preaching the gospel of the kingdom, and healing every sickness and every disease among the people."],
['Matthew','Matt','09','036','36',"But when he saw the multitudes, he was moved with compassion on them, because they fainted, and were scattered abroad, as sheep having no shepherd."],
['Matthew','Matt','09','037','37',"Then saith he unto his disciples, The harvest truly \emph{is} plenteous, but the labourers \emph{are} few;"],
['Matthew','Matt','09','038','38',"Pray ye therefore the Lord of the harvest, that he will send forth labourers into his harvest."]
]

Matthew10 = [
    ['Matthew','Matt','10','001','1',"And when he had called unto \emph{him} his twelve disciples, he gave them power \emph{against} unclean spirits, to cast them out, and to heal all manner of sickness and all manner of disease."],
['Matthew','Matt','10','002','2',"Now the names of the twelve apostles are these; The first, Simon, who is called Peter, and Andrew his brother; James \emph{the} \emph{son} of Zebedee, and John his brother;"],
['Matthew','Matt','10','003','3',"Philip, and Bartholomew; Thomas, and Matthew the publican; James \emph{the} \emph{son} of Alphaeus, and Lebbaeus, whose surname was Thaddaeus;"],
['Matthew','Matt','10','004','4',"Simon the Canaanite, and Judas Iscariot, who also betrayed him."],
['Matthew','Matt','10','005','5',"These twelve Jesus sent forth, and commanded them, saying, Go not into the way of the Gentiles, and into \emph{any} city of the Samaritans enter ye not:"],
['Matthew','Matt','10','006','6',"But go rather to the lost sheep of the house of Israel."],
['Matthew','Matt','10','007','7',"And as ye go, preach, saying, The kingdom of heaven is at hand."],
['Matthew','Matt','10','008','8',"Heal the sick, cleanse the lepers, raise the dead, cast out devils: freely ye have received, freely give."],
['Matthew','Matt','10','009','9',"Provide neither gold, nor silver, nor brass in your purses,"],
['Matthew','Matt','10','010','10',"Nor scrip for \emph{your} journey, neither two coats, neither shoes, nor yet staves: for the workman is worthy of his meat."],
['Matthew','Matt','10','011','11',"And into whatsoever city or town ye shall enter, enquire who in it is worthy; and there abide till ye go thence."],
['Matthew','Matt','10','012','12',"And when ye come into an house, salute it."],
['Matthew','Matt','10','013','13',"And if the house be worthy, let your peace come upon it: but if it be not worthy, let your peace return to you."],
['Matthew','Matt','10','014','14',"And whosoever shall not receive you, nor hear your words, when ye depart out of that house or city, shake off the dust of your feet."],
['Matthew','Matt','10','015','15',"Verily I say unto you, It shall be more tolerable for the land of Sodom and Gomorrha in the day of judgment, than for that city."],
['Matthew','Matt','10','016','16',"Behold, I send you forth as sheep in the midst of wolves: be ye therefore wise as serpents, and harmless as doves."],
['Matthew','Matt','10','017','17',"But beware of men: for they will deliver you up to the councils, and they will scourge you in their synagogues;"],
['Matthew','Matt','10','018','18',"And ye shall be brought before governors and kings for my sake, for a testimony against them and the Gentiles."],
['Matthew','Matt','10','019','19',"But when they deliver you up, take no thought how or what ye shall speak: for it shall be given you in that same hour what ye shall speak."],
['Matthew','Matt','10','020','20',"For it is not ye that speak, but the Spirit of your Father which speaketh in you."],
['Matthew','Matt','10','021','21',"And the brother shall deliver up the brother to death, and the father the child: and the children shall rise up against \emph{their} parents, and cause them to be put to death."],
['Matthew','Matt','10','022','22',"And ye shall be hated of all \emph{men} for my name's sake: but he that endureth to the end shall be saved."],
['Matthew','Matt','10','023','23',"But when they persecute you in this city, flee ye into another: for verily I say unto you, Ye shall not have gone over the cities of Israel, till the Son of man be come."],
['Matthew','Matt','10','024','24',"The disciple is not above \emph{his} master, nor the servant above his lord."],
['Matthew','Matt','10','025','25',"It is enough for the disciple that he be as his master, and the servant as his lord. If they have called the master of the house Beelzebub, how much more \emph{shall} \emph{they} \emph{call} them of his household?"],
['Matthew','Matt','10','026','26',"Fear them not therefore: for there is nothing covered, that shall not be revealed; and hid, that shall not be known."],
['Matthew','Matt','10','027','27',"What I tell you in darkness, \emph{that} speak ye in light: and what ye hear in the ear, \emph{that} preach ye upon the housetops."],
['Matthew','Matt','10','028','28',"And fear not them which kill the body, but are not able to kill the soul: but rather fear him which is able to destroy both soul and body in hell."],
['Matthew','Matt','10','029','29',"Are not two sparrows sold for a farthing? and one of them shall not fall on the ground without your Father."],
['Matthew','Matt','10','030','30',"But the very hairs of your head are all numbered."],
['Matthew','Matt','10','031','31',"Fear ye not therefore, ye are of more value than many sparrows."],
['Matthew','Matt','10','032','32',"Whosoever therefore shall confess me before men, him will I confess also before my Father which is in heaven."],
['Matthew','Matt','10','033','33',"But whosoever shall deny me before men, him will I also deny before my Father which is in heaven."],
['Matthew','Matt','10','034','34',"Think not that I am come to send peace on earth: I came not to send peace, but a sword."],
['Matthew','Matt','10','035','35',"For I am come to set a man at variance against his father, and the daughter against her mother, and the daughter in law against her mother in law."],
['Matthew','Matt','10','036','36',"And a man's foes \emph{shall} \emph{be} they of his own household."],
['Matthew','Matt','10','037','37',"He that loveth father or mother more than me is not worthy of me: and he that loveth son or daughter more than me is not worthy of me."],
['Matthew','Matt','10','038','38',"And he that taketh not his cross, and followeth after me, is not worthy of me."],
['Matthew','Matt','10','039','39',"He that findeth his life shall lose it: and he that loseth his life for my sake shall find it."],
['Matthew','Matt','10','040','40',"He that receiveth you receiveth me, and he that receiveth me receiveth him that sent me."],
['Matthew','Matt','10','041','41',"He that receiveth a prophet in the name of a prophet shall receive a prophet's reward; and he that receiveth a righteous man in the name of a righteous man shall receive a righteous man's reward."],
['Matthew','Matt','10','042','42',"And whosoever shall give to drink unto one of these little ones a cup of cold \emph{water} only in the name of a disciple, verily I say unto you, he shall in no wise lose his reward."]
]

Matthew11 = [
    ['Matthew','Matt','11','001','1',"And it came to pass, when Jesus had made an end of commanding his twelve disciples, he departed thence to teach and to preach in their cities."],
['Matthew','Matt','11','002','2',"Now when John had heard in the prison the works of Christ, he sent two of his disciples,"],
['Matthew','Matt','11','003','3',"And said unto him, Art thou he that should come, or do we look for another?"],
['Matthew','Matt','11','004','4',"Jesus answered and said unto them, Go and shew John again those things which ye do hear and see:"],
['Matthew','Matt','11','005','5',"The blind receive their sight, and the lame walk, the lepers are cleansed, and the deaf hear, the dead are raised up, and the poor have the gospel preached to them."],
['Matthew','Matt','11','006','6',"And blessed is \emph{he}, whosoever shall not be offended in me."],
['Matthew','Matt','11','007','7',"And as they departed, Jesus began to say unto the multitudes concerning John, What went ye out into the wilderness to see? A reed shaken with the wind?"],
['Matthew','Matt','11','008','8',"But what went ye out for to see? A man clothed in soft raiment? behold, they that wear soft \emph{clothing} are in kings' houses."],
['Matthew','Matt','11','009','9',"But what went ye out for to see? A prophet? yea, I say unto you, and more than a prophet."],
['Matthew','Matt','11','010','10',"For this is \emph{he}, of whom it is written, Behold, I send my messenger before thy face, which shall prepare thy way before thee."],
['Matthew','Matt','11','011','11',"Verily I say unto you, Among them that are born of women there hath not risen a greater than John the Baptist: notwithstanding he that is least in the kingdom of heaven is greater than he."],
['Matthew','Matt','11','012','12',"And from the days of John the Baptist until now the kingdom of heaven suffereth violence, and the violent take it by force."],
['Matthew','Matt','11','013','13',"For all the prophets and the law prophesied until John."],
['Matthew','Matt','11','014','14',"And if ye will receive \emph{it}, this is Elias, which was for to come."],
['Matthew','Matt','11','015','15',"He that hath ears to hear, let him hear."],
['Matthew','Matt','11','016','16',"But whereunto shall I liken this generation? It is like unto children sitting in the markets, and calling unto their fellows,"],
['Matthew','Matt','11','017','17',"And saying, We have piped unto you, and ye have not danced; we have mourned unto you, and ye have not lamented."],
['Matthew','Matt','11','018','18',"For John came neither eating nor drinking, and they say, He hath a devil."],
['Matthew','Matt','11','019','19',"The Son of man came eating and drinking, and they say, Behold a man gluttonous, and a winebibber, a friend of publicans and sinners. But wisdom is justified of her children."],
['Matthew','Matt','11','020','20',"Then began he to upbraid the cities wherein most of his mighty works were done, because they repented not:"],
['Matthew','Matt','11','021','21',"Woe unto thee, Chorazin! woe unto thee, Bethsaida! for if the mighty works, which were done in you, had been done in Tyre and Sidon, they would have repented long ago in sackcloth and ashes."],
['Matthew','Matt','11','022','22',"But I say unto you, It shall be more tolerable for Tyre and Sidon at the day of judgment, than for you."],
['Matthew','Matt','11','023','23',"And thou, Capernaum, which art exalted unto heaven, shalt be brought down to hell: for if the mighty works, which have been done in thee, had been done in Sodom, it would have remained until this day."],
['Matthew','Matt','11','024','24',"But I say unto you, That it shall be more tolerable for the land of Sodom in the day of judgment, than for thee."],
['Matthew','Matt','11','025','25',"At that time Jesus answered and said, I thank thee, O Father, Lord of heaven and earth, because thou hast hid these things from the wise and prudent, and hast revealed them unto babes."],
['Matthew','Matt','11','026','26',"Even so, Father: for so it seemed good in thy sight."],
['Matthew','Matt','11','027','27',"All things are delivered unto me of my Father: and no man knoweth the Son, but the Father; neither knoweth any man the Father, save the Son, and \emph{he} to whomsoever the Son will reveal \emph{him}."],
['Matthew','Matt','11','028','28',"Come unto me, all \emph{ye} that labour and are heavy laden, and I will give you rest."],
['Matthew','Matt','11','029','29',"Take my yoke upon you, and learn of me; for I am meek and lowly in heart: and ye shall find rest unto your souls."],
['Matthew','Matt','11','030','30',"For my yoke \emph{is} easy, and my burden is light."]
]

Matthew12 = [
    ['Matthew','Matt','12','001','1',"At that time Jesus went on the sabbath day through the corn; and his disciples were an hungred, and began to pluck the ears of corn, and to eat."],
['Matthew','Matt','12','002','2',"But when the Pharisees saw \emph{it}, they said unto him, Behold, thy disciples do that which is not lawful to do upon the sabbath day."],
['Matthew','Matt','12','003','3',"But he said unto them, Have ye not read what David did, when he was an hungred, and they that were with him;"],
['Matthew','Matt','12','004','4',"How he entered into the house of God, and did eat the shewbread, which was not lawful for him to eat, neither for them which were with him, but only for the priests?"],
['Matthew','Matt','12','005','5',"Or have ye not read in the law, how that on the sabbath days the priests in the temple profane the sabbath, and are blameless?"],
['Matthew','Matt','12','006','6',"But I say unto you, That in this place is \emph{one} greater than the temple."],
['Matthew','Matt','12','007','7',"But if ye had known what \emph{this} meaneth, I will have mercy, and not sacrifice, ye would not have condemned the guiltless."],
['Matthew','Matt','12','008','8',"For the Son of man is Lord even of the sabbath day."],
['Matthew','Matt','12','009','9',"And when he was departed thence, he went into their synagogue:"],
['Matthew','Matt','12','010','10',"And, behold, there was a man which had \emph{his} hand withered. And they asked him, saying, Is it lawful to heal on the sabbath days? that they might accuse him."],
['Matthew','Matt','12','011','11',"And he said unto them, What man shall there be among you, that shall have one sheep, and if it fall into a pit on the sabbath day, will he not lay hold on it, and lift \emph{it} out?"],
['Matthew','Matt','12','012','12',"How much then is a man better than a sheep? Wherefore it is lawful to do well on the sabbath days."],
['Matthew','Matt','12','013','13',"Then saith he to the man, Stretch forth thine hand. And he stretched \emph{it} forth; and it was restored whole, like as the other."],
['Matthew','Matt','12','014','14',"Then the Pharisees went out, and held a council against him, how they might destroy him."],
['Matthew','Matt','12','015','15',"But when Jesus knew \emph{it}, he withdrew himself from thence: and great multitudes followed him, and he healed them all;"],
['Matthew','Matt','12','016','16',"And charged them that they should not make him known:"],
['Matthew','Matt','12','017','17',"That it might be fulfilled which was spoken by Esaias the prophet, saying,"],
['Matthew','Matt','12','018','18',"Behold my servant, whom I have chosen; my beloved, in whom my soul is well pleased: I will put my spirit upon him, and he shall shew judgment to the Gentiles."],
['Matthew','Matt','12','019','19',"He shall not strive, nor cry; neither shall any man hear his voice in the streets."],
['Matthew','Matt','12','020','20',"A bruised reed shall he not break, and smoking flax shall he not quench, till he send forth judgment unto victory."],
['Matthew','Matt','12','021','21',"And in his name shall the Gentiles trust."],
['Matthew','Matt','12','022','22',"Then was brought unto him one possessed with a devil, blind, and dumb: and he healed him, insomuch that the blind and dumb both spake and saw."],
['Matthew','Matt','12','023','23',"And all the people were amazed, and said, Is not this the son of David?"],
['Matthew','Matt','12','024','24',"But when the Pharisees heard \emph{it}, they said, This \emph{fellow} doth not cast out devils, but by Beelzebub the prince of the devils."],
['Matthew','Matt','12','025','25',"And Jesus knew their thoughts, and said unto them, Every kingdom divided against itself is brought to desolation; and every city or house divided against itself shall not stand:"],
['Matthew','Matt','12','026','26',"And if Satan cast out Satan, he is divided against himself; how shall then his kingdom stand?"],
['Matthew','Matt','12','027','27',"And if I by Beelzebub cast out devils, by whom do your children cast \emph{them} out? therefore they shall be your judges."],
['Matthew','Matt','12','028','28',"But if I cast out devils by the Spirit of God, then the kingdom of God is come unto you."],
['Matthew','Matt','12','029','29',"Or else how can one enter into a strong man's house, and spoil his goods, except he first bind the strong man? and then he will spoil his house."],
['Matthew','Matt','12','030','30',"He that is not with me is against me; and he that gathereth not with me scattereth abroad."],
['Matthew','Matt','12','031','31',"Wherefore I say unto you, All manner of sin and blasphemy shall be forgiven unto men: but the blasphemy \emph{against} the \emph{Holy} Ghost shall not be forgiven unto men."],
['Matthew','Matt','12','032','32',"And whosoever speaketh a word against the Son of man, it shall be forgiven him: but whosoever speaketh against the Holy Ghost, it shall not be forgiven him, neither in this world, neither in the \emph{world} to come."],
['Matthew','Matt','12','033','33',"Either make the tree good, and his fruit good; or else make the tree corrupt, and his fruit corrupt: for the tree is known by \emph{his} fruit."],
['Matthew','Matt','12','034','34',"O generation of vipers, how can ye, being evil, speak good things? for out of the abundance of the heart the mouth speaketh."],
['Matthew','Matt','12','035','35',"A good man out of the good treasure of the heart bringeth forth good things: and an evil man out of the evil treasure bringeth forth evil things."],
['Matthew','Matt','12','036','36',"But I say unto you, That every idle word that men shall speak, they shall give account thereof in the day of judgment."],
['Matthew','Matt','12','037','37',"For by thy words thou shalt be justified, and by thy words thou shalt be condemned."],
['Matthew','Matt','12','038','38',"Then certain of the scribes and of the Pharisees answered, saying, Master, we would see a sign from thee."],
['Matthew','Matt','12','039','39',"But he answered and said unto them, An evil and adulterous generation seeketh after a sign; and there shall no sign be given to it, but the sign of the prophet Jonas:"],
['Matthew','Matt','12','040','40',"For as Jonas was three days and three nights in the whale's belly; so shall the Son of man be three days and three nights in the heart of the earth."],
['Matthew','Matt','12','041','41',"The men of Nineveh shall rise in judgment with this generation, and shall condemn it: because they repented at the preaching of Jonas; and, behold, a greater than Jonas \emph{is} here."],
['Matthew','Matt','12','042','42',"The queen of the south shall rise up in the judgment with this generation, and shall condemn it: for she came from the uttermost parts of the earth to hear the wisdom of Solomon; and, behold, a greater than Solomon \emph{is} here."],
['Matthew','Matt','12','043','43',"When the unclean spirit is gone out of a man, he walketh through dry places, seeking rest, and findeth none."],
['Matthew','Matt','12','044','44',"Then he saith, I will return into my house from whence I came out; and when he is come, he findeth \emph{it} empty, swept, and garnished."],
['Matthew','Matt','12','045','45',"Then goeth he, and taketh with himself seven other spirits more wicked than himself, and they enter in and dwell there: and the last \emph{state} of that man is worse than the first. Even so shall it be also unto this wicked generation."],
['Matthew','Matt','12','046','46',"While he yet talked to the people, behold, \emph{his} mother and his brethren stood without, desiring to speak with him."],
['Matthew','Matt','12','047','47',"Then one said unto him, Behold, thy mother and thy brethren stand without, desiring to speak with thee."],
['Matthew','Matt','12','048','48',"But he answered and said unto him that told him, Who is my mother? and who are my brethren?"],
['Matthew','Matt','12','049','49',"And he stretched forth his hand toward his disciples, and said, Behold my mother and my brethren!"],
['Matthew','Matt','12','050','50',"For whosoever shall do the will of my Father which is in heaven, the same is my brother, and sister, and mother."]
]

Matthew13 = [
    ['Matthew','Matt','13','001','1',"The same day went Jesus out of the house, and sat by the sea side."],
['Matthew','Matt','13','002','2',"And great multitudes were gathered together unto him, so that he went into a ship, and sat; and the whole multitude stood on the shore."],
['Matthew','Matt','13','003','3',"And he spake many things unto them in parables, saying, Behold, a sower went forth to sow;"],
['Matthew','Matt','13','004','4',"And when he sowed, some \emph{seeds} fell by the way side, and the fowls came and devoured them up:"],
['Matthew','Matt','13','005','5',"Some fell upon stony places, where they had not much earth: and forthwith they sprung up, because they had no deepness of earth:"],
['Matthew','Matt','13','006','6',"And when the sun was up, they were scorched; and because they had no root, they withered away."],
['Matthew','Matt','13','007','7',"And some fell among thorns; and the thorns sprung up, and choked them:"],
['Matthew','Matt','13','008','8',"But other fell into good ground, and brought forth fruit, some an hundredfold, some sixtyfold, some thirtyfold."],
['Matthew','Matt','13','009','9',"Who hath ears to hear, let him hear."],
['Matthew','Matt','13','010','10',"And the disciples came, and said unto him, Why speakest thou unto them in parables?"],
['Matthew','Matt','13','011','11',"He answered and said unto them, Because it is given unto you to know the mysteries of the kingdom of heaven, but to them it is not given."],
['Matthew','Matt','13','012','12',"For whosoever hath, to him shall be given, and he shall have more abundance: but whosoever hath not, from him shall be taken away even that he hath."],
['Matthew','Matt','13','013','13',"Therefore speak I to them in parables: because they seeing see not; and hearing they hear not, neither do they understand."],
['Matthew','Matt','13','014','14',"And in them is fulfilled the prophecy of Esaias, which saith, By hearing ye shall hear, and shall not understand; and seeing ye shall see, and shall not perceive:"],
['Matthew','Matt','13','015','15',"For this people's heart is waxed gross, and \emph{their} ears are dull of hearing, and their eyes they have closed; lest at any time they should see with \emph{their} eyes, and hear with \emph{their} ears, and should understand with \emph{their} heart, and should be converted, and I should heal them."],
['Matthew','Matt','13','016','16',"But blessed \emph{are} your eyes, for they see: and your ears, for they hear."],
['Matthew','Matt','13','017','17',"For verily I say unto you, That many prophets and righteous \emph{men} have desired to see \emph{those} \emph{things} which ye see, and have not seen \emph{them}; and to hear \emph{those} \emph{things} which ye hear, and have not heard \emph{them}."],
['Matthew','Matt','13','018','18',"Hear ye therefore the parable of the sower."],
['Matthew','Matt','13','019','19',"When any one heareth the word of the kingdom, and understandeth \emph{it} not, then cometh the wicked \emph{one}, and catcheth away that which was sown in his heart. This is he which received seed by the way side."],
['Matthew','Matt','13','020','20',"But he that received the seed into stony places, the same is he that heareth the word, and anon with joy receiveth it;"],
['Matthew','Matt','13','021','21',"Yet hath he not root in himself, but dureth for a while: for when tribulation or persecution ariseth because of the word, by and by he is offended."],
['Matthew','Matt','13','022','22',"He also that received seed among the thorns is he that heareth the word; and the care of this world, and the deceitfulness of riches, choke the word, and he becometh unfruitful."],
['Matthew','Matt','13','023','23',"But he that received seed into the good ground is he that heareth the word, and understandeth \emph{it}; which also beareth fruit, and bringeth forth, some an hundredfold, some sixty, some thirty."],
['Matthew','Matt','13','024','24',"Another parable put he forth unto them, saying, The kingdom of heaven is likened unto a man which sowed good seed in his field:"],
['Matthew','Matt','13','025','25',"But while men slept, his enemy came and sowed tares among the wheat, and went his way."],
['Matthew','Matt','13','026','26',"But when the blade was sprung up, and brought forth fruit, then appeared the tares also."],
['Matthew','Matt','13','027','27',"So the servants of the householder came and said unto him, Sir, didst not thou sow good seed in thy field? from whence then hath it tares?"],
['Matthew','Matt','13','028','28',"He said unto them, An enemy hath done this. The servants said unto him, Wilt thou then that we go and gather them up?"],
['Matthew','Matt','13','029','29',"But he said, Nay; lest while ye gather up the tares, ye root up also the wheat with them."],
['Matthew','Matt','13','030','30',"Let both grow together until the harvest: and in the time of harvest I will say to the reapers, Gather ye together first the tares, and bind them in bundles to burn them: but gather the wheat into my barn."],
['Matthew','Matt','13','031','31',"Another parable put he forth unto them, saying, The kingdom of heaven is like to a grain of mustard seed, which a man took, and sowed in his field:"],
['Matthew','Matt','13','032','32',"Which indeed is the least of all seeds: but when it is grown, it is the greatest among herbs, and becometh a tree, so that the birds of the air come and lodge in the branches thereof."],
['Matthew','Matt','13','033','33',"Another parable spake he unto them; The kingdom of heaven is like unto leaven, which a woman took, and hid in three measures of meal, till the whole was leavened."],
['Matthew','Matt','13','034','34',"All these things spake Jesus unto the multitude in parables; and without a parable spake he not unto them:"],
['Matthew','Matt','13','035','35',"That it might be fulfilled which was spoken by the prophet, saying, I will open my mouth in parables; I will utter things which have been kept secret from the foundation of the world."],
['Matthew','Matt','13','036','36',"Then Jesus sent the multitude away, and went into the house: and his disciples came unto him, saying, Declare unto us the parable of the tares of the field."],
['Matthew','Matt','13','037','37',"He answered and said unto them, He that soweth the good seed is the Son of man;"],
['Matthew','Matt','13','038','38',"The field is the world; the good seed are the children of the kingdom; but the tares are the children of the wicked \emph{one};"],
['Matthew','Matt','13','039','39',"The enemy that sowed them is the devil; the harvest is the end of the world; and the reapers are the angels."],
['Matthew','Matt','13','040','40',"As therefore the tares are gathered and burned in the fire; so shall it be in the end of this world."],
['Matthew','Matt','13','041','41',"The Son of man shall send forth his angels, and they shall gather out of his kingdom all things that offend, and them which do iniquity;"],
['Matthew','Matt','13','042','42',"And shall cast them into a furnace of fire: there shall be wailing and gnashing of teeth."],
['Matthew','Matt','13','043','43',"Then shall the righteous shine forth as the sun in the kingdom of their Father. Who hath ears to hear, let him hear."],
['Matthew','Matt','13','044','44',"Again, the kingdom of heaven is like unto treasure hid in a field; the which when a man hath found, he hideth, and for joy thereof goeth and selleth all that he hath, and buyeth that field."],
['Matthew','Matt','13','045','45',"Again, the kingdom of heaven is like unto a merchant man, seeking goodly pearls:"],
['Matthew','Matt','13','046','46',"Who, when he had found one pearl of great price, went and sold all that he had, and bought it."],
['Matthew','Matt','13','047','47',"Again, the kingdom of heaven is like unto a net, that was cast into the sea, and gathered of every kind:"],
['Matthew','Matt','13','048','48',"Which, when it was full, they drew to shore, and sat down, and gathered the good into vessels, but cast the bad away."],
['Matthew','Matt','13','049','49',"So shall it be at the end of the world: the angels shall come forth, and sever the wicked from among the just,"],
['Matthew','Matt','13','050','50',"And shall cast them into the furnace of fire: there shall be wailing and gnashing of teeth."],
['Matthew','Matt','13','051','51',"Jesus saith unto them, Have ye understood all these things? They say unto him, Yea, Lord."],
['Matthew','Matt','13','052','52',"Then said he unto them, Therefore every scribe \emph{which} \emph{is} instructed unto the kingdom of heaven is like unto a man \emph{that} \emph{is} an householder, which bringeth forth out of his treasure \emph{things} new and old."],
['Matthew','Matt','13','053','53',"And it came to pass, \emph{that} when Jesus had finished these parables, he departed thence."],
['Matthew','Matt','13','054','54',"And when he was come into his own country, he taught them in their synagogue, insomuch that they were astonished, and said, Whence hath this \emph{man} this wisdom, and \emph{these} mighty works?"],
['Matthew','Matt','13','055','55',"Is not this the carpenter's son? is not his mother called Mary? and his brethren, James, and Joses, and Simon, and Judas?"],
['Matthew','Matt','13','056','56',"And his sisters, are they not all with us? Whence then hath this \emph{man} all these things?"],
['Matthew','Matt','13','057','57',"And they were offended in him. But Jesus said unto them, A prophet is not without honour, save in his own country, and in his own house."],
['Matthew','Matt','13','058','58',"And he did not many mighty works there because of their unbelief."]
]

Matthew14 = [
['Matthew','Matt','14','001','1',"At that time Herod the tetrarch heard of the fame of Jesus,"],
['Matthew','Matt','14','002','2',"And said unto his servants, This is John the Baptist; he is risen from the dead; and therefore mighty works do shew forth themselves in him."],
['Matthew','Matt','14','003','3',"For Herod had laid hold on John, and bound him, and put \emph{him} in prison for Herodias' sake, his brother Philip's wife."],
['Matthew','Matt','14','004','4',"For John said unto him, It is not lawful for thee to have her."],
['Matthew','Matt','14','005','5',"And when he would have put him to death, he feared the multitude, because they counted him as a prophet."],
['Matthew','Matt','14','006','6',"But when Herod's birthday was kept, the daughter of Herodias danced before them, and pleased Herod."],
['Matthew','Matt','14','007','7',"Whereupon he promised with an oath to give her whatsoever she would ask."],
['Matthew','Matt','14','008','8',"And she, being before instructed of her mother, said, Give me here John Baptist's head in a charger."],
['Matthew','Matt','14','009','9',"And the king was sorry: nevertheless for the oath's sake, and them which sat with him at meat, he commanded \emph{it} to be given \emph{her}."],
['Matthew','Matt','14','010','10',"And he sent, and beheaded John in the prison."],
['Matthew','Matt','14','011','11',"And his head was brought in a charger, and given to the damsel: and she brought \emph{it} to her mother."],
['Matthew','Matt','14','012','12',"And his disciples came, and took up the body, and buried it, and went and told Jesus."],
['Matthew','Matt','14','013','13',"When Jesus heard \emph{of} \emph{it}, he departed thence by ship into a desert place apart: and when the people had heard \emph{thereof}, they followed him on foot out of the cities."],
['Matthew','Matt','14','014','14',"And Jesus went forth, and saw a great multitude, and was moved with compassion toward them, and he healed their sick."],
['Matthew','Matt','14','015','15',"And when it was evening, his disciples came to him, saying, This is a desert place, and the time is now past; send the multitude away, that they may go into the villages, and buy themselves victuals."],
['Matthew','Matt','14','016','16',"But Jesus said unto them, They need not depart; give ye them to eat."],
['Matthew','Matt','14','017','17',"And they say unto him, We have here but five loaves, and two fishes."],
['Matthew','Matt','14','018','18',"He said, Bring them hither to me."],
['Matthew','Matt','14','019','19',"And he commanded the multitude to sit down on the grass, and took the five loaves, and the two fishes, and looking up to heaven, he blessed, and brake, and gave the loaves to \emph{his} disciples, and the disciples to the multitude."],
['Matthew','Matt','14','020','20',"And they did all eat, and were filled: and they took up of the fragments that remained twelve baskets full."],
['Matthew','Matt','14','021','21',"And they that had eaten were about five thousand men, beside women and children."],
['Matthew','Matt','14','022','22',"And straightway Jesus constrained his disciples to get into a ship, and to go before him unto the other side, while he sent the multitudes away."],
['Matthew','Matt','14','023','23',"And when he had sent the multitudes away, he went up into a mountain apart to pray: and when the evening was come, he was there alone."],
['Matthew','Matt','14','024','24',"But the ship was now in the midst of the sea, tossed with waves: for the wind was contrary."],
['Matthew','Matt','14','025','25',"And in the fourth watch of the night Jesus went unto them, walking on the sea."],
['Matthew','Matt','14','026','26',"And when the disciples saw him walking on the sea, they were troubled, saying, It is a spirit; and they cried out for fear."],
['Matthew','Matt','14','027','27',"But straightway Jesus spake unto them, saying, Be of good cheer; it is I; be not afraid."],
['Matthew','Matt','14','028','28',"And Peter answered him and said, Lord, if it be thou, bid me come unto thee on the water."],
['Matthew','Matt','14','029','29',"And he said, Come. And when Peter was come down out of the ship, he walked on the water, to go to Jesus."],
['Matthew','Matt','14','030','30',"But when he saw the wind boisterous, he was afraid; and beginning to sink, he cried, saying, Lord, save me."],
['Matthew','Matt','14','031','31',"And immediately Jesus stretched forth \emph{his} hand, and caught him, and said unto him, O thou of little faith, wherefore didst thou doubt?"],
['Matthew','Matt','14','032','32',"And when they were come into the ship, the wind ceased."],
['Matthew','Matt','14','033','33',"Then they that were in the ship came and worshipped him, saying, Of a truth thou art the Son of God."],
['Matthew','Matt','14','034','34',"And when they were gone over, they came into the land of Gennesaret."],
['Matthew','Matt','14','035','35',"And when the men of that place had knowledge of him, they sent out into all that country round about, and brought unto him all that were diseased;"],
['Matthew','Matt','14','036','36',"And besought him that they might only touch the hem of his garment: and as many as touched were made perfectly whole."]
]

Matthew15 = [
    ['Matthew','Matt','15','001','1',"Then came to Jesus scribes and Pharisees, which were of Jerusalem, saying,"],
['Matthew','Matt','15','002','2',"Why do thy disciples transgress the tradition of the elders? for they wash not their hands when they eat bread."],
['Matthew','Matt','15','003','3',"But he answered and said unto them, Why do ye also transgress the commandment of God by your tradition?"],
['Matthew','Matt','15','004','4',"For God commanded, saying, Honour thy father and mother: and, He that curseth father or mother, let him die the death."],
['Matthew','Matt','15','005','5',"But ye say, Whosoever shall say to \emph{his} father or \emph{his} mother, \emph{It} \emph{is} a gift, by whatsoever thou mightest be profited by me;"],
['Matthew','Matt','15','006','6',"And honour not his father or his mother, \emph{he} \emph{shall} \emph{be} \emph{free}. Thus have ye made the commandment of God of none effect by your tradition."],
['Matthew','Matt','15','007','7',"\emph{Ye} hypocrites, well did Esaias prophesy of you, saying,"],
['Matthew','Matt','15','008','8',"This people draweth nigh unto me with their mouth, and honoureth me with \emph{their} lips; but their heart is far from me."],
['Matthew','Matt','15','009','9',"But in vain they do worship me, teaching \emph{for} doctrines the commandments of men."],
['Matthew','Matt','15','010','10',"And he called the multitude, and said unto them, Hear, and understand:"],
['Matthew','Matt','15','011','11',"Not that which goeth into the mouth defileth a man; but that which cometh out of the mouth, this defileth a man."],
['Matthew','Matt','15','012','12',"Then came his disciples, and said unto him, Knowest thou that the Pharisees were offended, after they heard this saying?"],
['Matthew','Matt','15','013','13',"But he answered and said, Every plant, which my heavenly Father hath not planted, shall be rooted up."],
['Matthew','Matt','15','014','14',"Let them alone: they be blind leaders of the blind. And if the blind lead the blind, both shall fall into the ditch."],
['Matthew','Matt','15','015','15',"Then answered Peter and said unto him, Declare unto us this parable."],
['Matthew','Matt','15','016','16',"And Jesus said, Are ye also yet without understanding?"],
['Matthew','Matt','15','017','17',"Do not ye yet understand, that whatsoever entereth in at the mouth goeth into the belly, and is cast out into the draught?"],
['Matthew','Matt','15','018','18',"But those things which proceed out of the mouth come forth from the heart; and they defile the man."],
['Matthew','Matt','15','019','19',"For out of the heart proceed evil thoughts, murders, adulteries, fornications, thefts, false witness, blasphemies:"],
['Matthew','Matt','15','020','20',"These are \emph{the} \emph{things} which defile a man: but to eat with unwashen hands defileth not a man."],
['Matthew','Matt','15','021','21',"Then Jesus went thence, and departed into the coasts of Tyre and Sidon."],
['Matthew','Matt','15','022','22',"And, behold, a woman of Canaan came out of the same coasts, and cried unto him, saying, Have mercy on me, O Lord, \emph{thou} Son of David; my daughter is grievously vexed with a devil."],
['Matthew','Matt','15','023','23',"But he answered her not a word. And his disciples came and besought him, saying, Send her away; for she crieth after us."],
['Matthew','Matt','15','024','24',"But he answered and said, I am not sent but unto the lost sheep of the house of Israel."],
['Matthew','Matt','15','025','25',"Then came she and worshipped him, saying, Lord, help me."],
['Matthew','Matt','15','026','26',"But he answered and said, It is not meet to take the children's bread, and to cast \emph{it} to dogs."],
['Matthew','Matt','15','027','27',"And she said, Truth, Lord: yet the dogs eat of the crumbs which fall from their masters' table."],
['Matthew','Matt','15','028','28',"Then Jesus answered and said unto her, O woman, great \emph{is} thy faith: be it unto thee even as thou wilt. And her daughter was made whole from that very hour."],
['Matthew','Matt','15','029','29',"And Jesus departed from thence, and came nigh unto the sea of Galilee; and went up into a mountain, and sat down there."],
['Matthew','Matt','15','030','30',"And great multitudes came unto him, having with them \emph{those} \emph{that} \emph{were} lame, blind, dumb, maimed, and many others, and cast them down at Jesus' feet; and he healed them:"],
['Matthew','Matt','15','031','31',"Insomuch that the multitude wondered, when they saw the dumb to speak, the maimed to be whole, the lame to walk, and the blind to see: and they glorified the God of Israel."],
['Matthew','Matt','15','032','32',"Then Jesus called his disciples \emph{unto} \emph{him}, and said, I have compassion on the multitude, because they continue with me now three days, and have nothing to eat: and I will not send them away fasting, lest they faint in the way."],
['Matthew','Matt','15','033','33',"And his disciples say unto him, Whence should we have so much bread in the wilderness, as to fill so great a multitude?"],
['Matthew','Matt','15','034','34',"And Jesus saith unto them, How many loaves have ye? And they said, Seven, and a few little fishes."],
['Matthew','Matt','15','035','35',"And he commanded the multitude to sit down on the ground."],
['Matthew','Matt','15','036','36',"And he took the seven loaves and the fishes, and gave thanks, and brake \emph{them}, and gave to his disciples, and the disciples to the multitude."],
['Matthew','Matt','15','037','37',"And they did all eat, and were filled: and they took up of the broken \emph{meat} that was left seven baskets full."],
['Matthew','Matt','15','038','38',"And they that did eat were four thousand men, beside women and children."],
['Matthew','Matt','15','039','39',"And he sent away the multitude, and took ship, and came into the coasts of Magdala."]
]

Matthew16 = [
    ['Matthew','Matt','16','001','1',"The Pharisees also with the Sadducees came, and tempting desired him that he would shew them a sign from heaven."],
['Matthew','Matt','16','002','2',"He answered and said unto them, When it is evening, ye say, \emph{It} \emph{will} \emph{be} fair weather: for the sky is red."],
['Matthew','Matt','16','003','3',"And in the morning, \emph{It} \emph{will} \emph{be} foul weather to day: for the sky is red and lowring. O \emph{ye} hypocrites, ye can discern the face of the sky; but can ye not \emph{discern} the signs of the times?"],
['Matthew','Matt','16','004','4',"A wicked and adulterous generation seeketh after a sign; and there shall no sign be given unto it, but the sign of the prophet Jonas. And he left them, and departed."],
['Matthew','Matt','16','005','5',"And when his disciples were come to the other side, they had forgotten to take bread."],
['Matthew','Matt','16','006','6',"Then Jesus said unto them, Take heed and beware of the leaven of the Pharisees and of the Sadducees."],
['Matthew','Matt','16','007','7',"And they reasoned among themselves, saying, \emph{It} \emph{is} because we have taken no bread."],
['Matthew','Matt','16','008','8',"\emph{Which} when Jesus perceived, he said unto them, O ye of little faith, why reason ye among yourselves, because ye have brought no bread?"],
['Matthew','Matt','16','009','9',"Do ye not yet understand, neither remember the five loaves of the five thousand, and how many baskets ye took up?"],
['Matthew','Matt','16','010','10',"Neither the seven loaves of the four thousand, and how many baskets ye took up?"],
['Matthew','Matt','16','011','11',"How is it that ye do not understand that I spake \emph{it} not to you concerning bread, that ye should beware of the leaven of the Pharisees and of the Sadducees?"],
['Matthew','Matt','16','012','12',"Then understood they how that he bade \emph{them} not beware of the leaven of bread, but of the doctrine of the Pharisees and of the Sadducees."],
['Matthew','Matt','16','013','13',"When Jesus came into the coasts of Caesarea Philippi, he asked his disciples, saying, Whom do men say that I the Son of man am?"],
['Matthew','Matt','16','014','14',"And they said, Some \emph{say} \emph{that} \emph{thou} \emph{art} John the Baptist: some, Elias; and others, Jeremias, or one of the prophets."],
['Matthew','Matt','16','015','15',"He saith unto them, But whom say ye that I am?"],
['Matthew','Matt','16','016','16',"And Simon Peter answered and said, Thou art the Christ, the Son of the living God."],
['Matthew','Matt','16','017','17',"And Jesus answered and said unto him, Blessed art thou, Simon Bar-jona: for flesh and blood hath not revealed \emph{it} unto thee, but my Father which is in heaven."],
['Matthew','Matt','16','018','18',"And I say also unto thee, That thou art Peter, and upon this rock I will build my church; and the gates of hell shall not prevail against it."],
['Matthew','Matt','16','019','19',"And I will give unto thee the keys of the kingdom of heaven: and whatsoever thou shalt bind on earth shall be bound in heaven: and whatsoever thou shalt loose on earth shall be loosed in heaven."],
['Matthew','Matt','16','020','20',"Then charged he his disciples that they should tell no man that he was Jesus the Christ."],
['Matthew','Matt','16','021','21',"From that time forth began Jesus to shew unto his disciples, how that he must go unto Jerusalem, and suffer many things of the elders and chief priests and scribes, and be killed, and be raised again the third day."],
['Matthew','Matt','16','022','22',"Then Peter took him, and began to rebuke him, saying, Be it far from thee, Lord: this shall not be unto thee."],
['Matthew','Matt','16','023','23',"But he turned, and said unto Peter, Get thee behind me, Satan: thou art an offence unto me: for thou savourest not the things that be of God, but those that be of men."],
['Matthew','Matt','16','024','24',"Then said Jesus unto his disciples, If any \emph{man} will come after me, let him deny himself, and take up his cross, and follow me."],
['Matthew','Matt','16','025','25',"For whosoever will save his life shall lose it: and whosoever will lose his life for my sake shall find it."],
['Matthew','Matt','16','026','26',"For what is a man profited, if he shall gain the whole world, and lose his own soul? or what shall a man give in exchange for his soul?"],
['Matthew','Matt','16','027','27',"For the Son of man shall come in the glory of his Father with his angels; and then he shall reward every man according to his works."],
['Matthew','Matt','16','028','28',"Verily I say unto you, There be some standing here, which shall not taste of death, till they see the Son of man coming in his kingdom."]
]

Matthew17 = [
    ['Matthew','Matt','17','001','1',"And after six days Jesus taketh Peter, James, and John his brother, and bringeth them up into an high mountain apart,"],
['Matthew','Matt','17','002','2',"And was transfigured before them: and his face did shine as the sun, and his raiment was white as the light."],
['Matthew','Matt','17','003','3',"And, behold, there appeared unto them Moses and Elias talking with him."],
['Matthew','Matt','17','004','4',"Then answered Peter, and said unto Jesus, Lord, it is good for us to be here: if thou wilt, let us make here three tabernacles; one for thee, and one for Moses, and one for Elias."],
['Matthew','Matt','17','005','5',"While he yet spake, behold, a bright cloud overshadowed them: and behold a voice out of the cloud, which said, This is my beloved Son, in whom I am well pleased; hear ye him."],
['Matthew','Matt','17','006','6',"And when the disciples heard \emph{it}, they fell on their face, and were sore afraid."],
['Matthew','Matt','17','007','7',"And Jesus came and touched them, and said, Arise, and be not afraid."],
['Matthew','Matt','17','008','8',"And when they had lifted up their eyes, they saw no man, save Jesus only."],
['Matthew','Matt','17','009','9',"And as they came down from the mountain, Jesus charged them, saying, Tell the vision to no man, until the Son of man be risen again from the dead."],
['Matthew','Matt','17','010','10',"And his disciples asked him, saying, Why then say the scribes that Elias must first come?"],
['Matthew','Matt','17','011','11',"And Jesus answered and said unto them, Elias truly shall first come, and restore all things."],
['Matthew','Matt','17','012','12',"But I say unto you, That Elias is come already, and they knew him not, but have done unto him whatsoever they listed. Likewise shall also the Son of man suffer of them."],
['Matthew','Matt','17','013','13',"Then the disciples understood that he spake unto them of John the Baptist."],
['Matthew','Matt','17','014','14',"And when they were come to the multitude, there came to him a \emph{certain} man, kneeling down to him, and saying,"],
['Matthew','Matt','17','015','15',"Lord, have mercy on my son: for he is lunatick, and sore vexed: for ofttimes he falleth into the fire, and oft into the water."],
['Matthew','Matt','17','016','16',"And I brought him to thy disciples, and they could not cure him."],
['Matthew','Matt','17','017','17',"Then Jesus answered and said, O faithless and perverse generation, how long shall I be with you? how long shall I suffer you? bring him hither to me."],
['Matthew','Matt','17','018','18',"And Jesus rebuked the devil; and he departed out of him: and the child was cured from that very hour."],
['Matthew','Matt','17','019','19',"Then came the disciples to Jesus apart, and said, Why could not we cast him out?"],
['Matthew','Matt','17','020','20',"And Jesus said unto them, Because of your unbelief: for verily I say unto you, If ye have faith as a grain of mustard seed, ye shall say unto this mountain, Remove hence to yonder place; and it shall remove; and nothing shall be impossible unto you."],
['Matthew','Matt','17','021','21',"Howbeit this kind goeth not out but by prayer and fasting."],
['Matthew','Matt','17','022','22',"And while they abode in Galilee, Jesus said unto them, The Son of man shall be betrayed into the hands of men:"],
['Matthew','Matt','17','023','23',"And they shall kill him, and the third day he shall be raised again. And they were exceeding sorry."],
['Matthew','Matt','17','024','24',"And when they were come to Capernaum, they that received tribute \emph{money} came to Peter, and said, Doth not your master pay tribute?"],
['Matthew','Matt','17','025','25',"He saith, Yes. And when he was come into the house, Jesus prevented him, saying, What thinkest thou, Simon? of whom do the kings of the earth take custom or tribute? of their own children, or of strangers?"],
['Matthew','Matt','17','026','26',"Peter saith unto him, Of strangers. Jesus saith unto him, Then are the children free."],
['Matthew','Matt','17','027','27',"Notwithstanding, lest we should offend them, go thou to the sea, and cast an hook, and take up the fish that first cometh up; and when thou hast opened his mouth, thou shalt find a piece of money: that take, and give unto them for me and thee."]
]

Matthew18 = [
    ['Matthew','Matt','18','001','1',"At the same time came the disciples unto Jesus, saying, Who is the greatest in the kingdom of heaven?"],
['Matthew','Matt','18','002','2',"And Jesus called a little child unto him, and set him in the midst of them,"],
['Matthew','Matt','18','003','3',"And said, Verily I say unto you, Except ye be converted, and become as little children, ye shall not enter into the kingdom of heaven."],
['Matthew','Matt','18','004','4',"Whosoever therefore shall humble himself as this little child, the same is greatest in the kingdom of heaven."],
['Matthew','Matt','18','005','5',"And whoso shall receive one such little child in my name receiveth me."],
['Matthew','Matt','18','006','6',"But whoso shall offend one of these little ones which believe in me, it were better for him that a millstone were hanged about his neck, and \emph{that} he were drowned in the depth of the sea."],
['Matthew','Matt','18','007','7',"Woe unto the world because of offences! for it must needs be that offences come; but woe to that man by whom the offence cometh!"],
['Matthew','Matt','18','008','8',"Wherefore if thy hand or thy foot offend thee, cut them off, and cast \emph{them} from thee: it is better for thee to enter into life halt or maimed, rather than having two hands or two feet to be cast into everlasting fire."],
['Matthew','Matt','18','009','9',"And if thine eye offend thee, pluck it out, and cast \emph{it} from thee: it is better for thee to enter into life with one eye, rather than having two eyes to be cast into hell fire."],
['Matthew','Matt','18','010','10',"Take heed that ye despise not one of these little ones; for I say unto you, That in heaven their angels do always behold the face of my Father which is in heaven."],
['Matthew','Matt','18','011','11',"For the Son of man is come to save that which was lost."],
['Matthew','Matt','18','012','12',"How think ye? if a man have an hundred sheep, and one of them be gone astray, doth he not leave the ninety and nine, and goeth into the mountains, and seeketh that which is gone astray?"],
['Matthew','Matt','18','013','13',"And if so be that he find it, verily I say unto you, he rejoiceth more of that \emph{sheep}, than of the ninety and nine which went not astray."],
['Matthew','Matt','18','014','14',"Even so it is not the will of your Father which is in heaven, that one of these little ones should perish."],
['Matthew','Matt','18','015','15',"Moreover if thy brother shall trespass against thee, go and tell him his fault between thee and him alone: if he shall hear thee, thou hast gained thy brother."],
['Matthew','Matt','18','016','16',"But if he will not hear \emph{thee,} \emph{then} take with thee one or two more, that in the mouth of two or three witnesses every word may be established."],
['Matthew','Matt','18','017','17',"And if he shall neglect to hear them, tell \emph{it} unto the church: but if he neglect to hear the church, let him be unto thee as an heathen man and a publican."],
['Matthew','Matt','18','018','18',"Verily I say unto you, Whatsoever ye shall bind on earth shall be bound in heaven: and whatsoever ye shall loose on earth shall be loosed in heaven."],
['Matthew','Matt','18','019','19',"Again I say unto you, That if two of you shall agree on earth as touching any thing that they shall ask, it shall be done for them of my Father which is in heaven."],
['Matthew','Matt','18','020','20',"For where two or three are gathered together in my name, there am I in the midst of them."],
['Matthew','Matt','18','021','21',"Then came Peter to him, and said, Lord, how oft shall my brother sin against me, and I forgive him? till seven times?"],
['Matthew','Matt','18','022','22',"Jesus saith unto him, I say not unto thee, Until seven times: but, Until seventy times seven."],
['Matthew','Matt','18','023','23',"Therefore is the kingdom of heaven likened unto a certain king, which would take account of his servants."],
['Matthew','Matt','18','024','24',"And when he had begun to reckon, one was brought unto him, which owed him ten thousand talents."],
['Matthew','Matt','18','025','25',"But forasmuch as he had not to pay, his lord commanded him to be sold, and his wife, and children, and all that he had, and payment to be made."],
['Matthew','Matt','18','026','26',"The servant therefore fell down, and worshipped him, saying, Lord, have patience with me, and I will pay thee all."],
['Matthew','Matt','18','027','27',"Then the lord of that servant was moved with compassion, and loosed him, and forgave him the debt."],
['Matthew','Matt','18','028','28',"But the same servant went out, and found one of his fellowservants, which owed him an hundred pence: and he laid hands on him, and took \emph{him} by the throat, saying, Pay me that thou owest."],
['Matthew','Matt','18','029','29',"And his fellowservant fell down at his feet, and besought him, saying, Have patience with me, and I will pay thee all."],
['Matthew','Matt','18','030','30',"And he would not: but went and cast him into prison, till he should pay the debt."],
['Matthew','Matt','18','031','31',"So when his fellowservants saw what was done, they were very sorry, and came and told unto their lord all that was done."],
['Matthew','Matt','18','032','32',"Then his lord, after that he had called him, said unto him, O thou wicked servant, I forgave thee all that debt, because thou desiredst me:"],
['Matthew','Matt','18','033','33',"Shouldest not thou also have had compassion on thy fellowservant, even as I had pity on thee?"],
['Matthew','Matt','18','034','34',"And his lord was wroth, and delivered him to the tormentors, till he should pay all that was due unto him."],
['Matthew','Matt','18','035','35',"So likewise shall my heavenly Father do also unto you, if ye from your hearts forgive not every one his brother their trespasses."]
]

Matthew19 = [
    ['Matthew','Matt','19','001','1',"And it came to pass, \emph{that} when Jesus had finished these sayings, he departed from Galilee, and came into the coasts of Judaea beyond Jordan;"],
['Matthew','Matt','19','002','2',"And great multitudes followed him; and he healed them there."],
['Matthew','Matt','19','003','3',"The Pharisees also came unto him, tempting him, and saying unto him, Is it lawful for a man to put away his wife for every cause?"],
['Matthew','Matt','19','004','4',"And he answered and said unto them, Have ye not read, that he which made \emph{them} at the beginning made them male and female,"],
['Matthew','Matt','19','005','5',"And said, For this cause shall a man leave father and mother, and shall cleave to his wife: and they twain shall be one flesh?"],
['Matthew','Matt','19','006','6',"Wherefore they are no more twain, but one flesh. What therefore God hath joined together, let not man put asunder."],
['Matthew','Matt','19','007','7',"They say unto him, Why did Moses then command to give a writing of divorcement, and to put her away?"],
['Matthew','Matt','19','008','8',"He saith unto them, Moses because of the hardness of your hearts suffered you to put away your wives: but from the beginning it was not so."],
['Matthew','Matt','19','009','9',"And I say unto you, Whosoever shall put away his wife, except \emph{it} \emph{be} for fornication, and shall marry another, committeth adultery: and whoso marrieth her which is put away doth commit adultery."],
['Matthew','Matt','19','010','10',"His disciples say unto him, If the case of the man be so with \emph{his} wife, it is not good to marry."],
['Matthew','Matt','19','011','11',"But he said unto them, All \emph{men} cannot receive this saying, save \emph{they} to whom it is given."],
['Matthew','Matt','19','012','12',"For there are some eunuchs, which were so born from \emph{their} mother's womb: and there are some eunuchs, which were made eunuchs of men: and there be eunuchs, which have made themselves eunuchs for the kingdom of heaven's sake. He that is able to receive \emph{it}, let him receive \emph{it}."],
['Matthew','Matt','19','013','13',"Then were there brought unto him little children, that he should put \emph{his} hands on them, and pray: and the disciples rebuked them."],
['Matthew','Matt','19','014','14',"But Jesus said, Suffer little children, and forbid them not, to come unto me: for of such is the kingdom of heaven."],
['Matthew','Matt','19','015','15',"And he laid \emph{his} hands on them, and departed thence."],
['Matthew','Matt','19','016','16',"And, behold, one came and said unto him, Good Master, what good thing shall I do, that I may have eternal life?"],
['Matthew','Matt','19','017','17',"And he said unto him, Why callest thou me good? \emph{there} \emph{is} none good but one, \emph{that} \emph{is}, God: but if thou wilt enter into life, keep the commandments."],
['Matthew','Matt','19','018','18',"He saith unto him, Which? Jesus said, Thou shalt do no murder, Thou shalt not commit adultery, Thou shalt not steal, Thou shalt not bear false witness,"],
['Matthew','Matt','19','019','19',"Honour thy father and \emph{thy} mother: and, Thou shalt love thy neighbour as thyself."],
['Matthew','Matt','19','020','20',"The young man saith unto him, All these things have I kept from my youth up: what lack I yet?"],
['Matthew','Matt','19','021','21',"Jesus said unto him, If thou wilt be perfect, go \emph{and} sell that thou hast, and give to the poor, and thou shalt have treasure in heaven: and come \emph{and} follow me."],
['Matthew','Matt','19','022','22',"But when the young man heard that saying, he went away sorrowful: for he had great possessions."],
['Matthew','Matt','19','023','23',"Then said Jesus unto his disciples, Verily I say unto you, That a rich man shall hardly enter into the kingdom of heaven."],
['Matthew','Matt','19','024','24',"And again I say unto you, It is easier for a camel to go through the eye of a needle, than for a rich man to enter into the kingdom of God."],
['Matthew','Matt','19','025','25',"When his disciples heard \emph{it}, they were exceedingly amazed, saying, Who then can be saved?"],
['Matthew','Matt','19','026','26',"But Jesus beheld \emph{them}, and said unto them, With men this is impossible; but with God all things are possible."],
['Matthew','Matt','19','027','27',"Then answered Peter and said unto him, Behold, we have forsaken all, and followed thee; what shall we have therefore?"],
['Matthew','Matt','19','028','28',"And Jesus said unto them, Verily I say unto you, That ye which have followed me, in the regeneration when the Son of man shall sit in the throne of his glory, ye also shall sit upon twelve thrones, judging the twelve tribes of Israel."],
['Matthew','Matt','19','029','29',"And every one that hath forsaken houses, or brethren, or sisters, or father, or mother, or wife, or children, or lands, for my name's sake, shall receive an hundredfold, and shall inherit everlasting life."],
['Matthew','Matt','19','030','30',"But many \emph{that} \emph{are} first shall be last; and the last \emph{shall} \emph{be} first."]
]

Matthew20 = [
             ['Matthew','Matt','20','001','1',"For the kingdom of heaven is like unto a man \emph{that} \emph{is} an householder, which went out early in the morning to hire labourers into his vineyard."],
['Matthew','Matt','20','002','2',"And when he had agreed with the labourers for a penny a day, he sent them into his vineyard."],
['Matthew','Matt','20','003','3',"And he went out about the third hour, and saw others standing idle in the marketplace,"],
['Matthew','Matt','20','004','4',"And said unto them; Go ye also into the vineyard, and whatsoever is right I will give you. And they went their way."],
['Matthew','Matt','20','005','5',"Again he went out about the sixth and ninth hour, and did likewise."],
['Matthew','Matt','20','006','6',"And about the eleventh hour he went out, and found others standing idle, and saith unto them, Why stand ye here all the day idle?"],
['Matthew','Matt','20','007','7',"They say unto him, Because no man hath hired us. He saith unto them, Go ye also into the vineyard; and whatsoever is right, \emph{that} shall ye receive."],
['Matthew','Matt','20','008','8',"So when even was come, the lord of the vineyard saith unto his steward, Call the labourers, and give them \emph{their} hire, beginning from the last unto the first."],
['Matthew','Matt','20','009','9',"And when they came that \emph{were} \emph{hired} about the eleventh hour, they received every man a penny."],
['Matthew','Matt','20','010','10',"But when the first came, they supposed that they should have received more; and they likewise received every man a penny."],
['Matthew','Matt','20','011','11',"And when they had received \emph{it}, they murmured against the goodman of the house,"],
['Matthew','Matt','20','012','12',"Saying, These last have wrought \emph{but} one hour, and thou hast made them equal unto us, which have borne the burden and heat of the day."],
['Matthew','Matt','20','013','13',"But he answered one of them, and said, Friend, I do thee no wrong: didst not thou agree with me for a penny?"],
['Matthew','Matt','20','014','14',"Take \emph{that} thine \emph{is}, and go thy way: I will give unto this last, even as unto thee."],
['Matthew','Matt','20','015','15',"Is it not lawful for me to do what I will with mine own? Is thine eye evil, because I am good?"],
['Matthew','Matt','20','016','16',"So the last shall be first, and the first last: for many be called, but few chosen."],
['Matthew','Matt','20','017','17',"And Jesus going up to Jerusalem took the twelve disciples apart in the way, and said unto them,"],
['Matthew','Matt','20','018','18',"Behold, we go up to Jerusalem; and the Son of man shall be betrayed unto the chief priests and unto the scribes, and they shall condemn him to death,"],
['Matthew','Matt','20','019','19',"And shall deliver him to the Gentiles to mock, and to scourge, and to crucify \emph{him}: and the third day he shall rise again."],
['Matthew','Matt','20','020','20',"Then came to him the mother of Zebedee's children with her sons, worshipping \emph{him}, and desiring a certain thing of him."],
['Matthew','Matt','20','021','21',"And he said unto her, What wilt thou? She saith unto him, Grant that these my two sons may sit, the one on thy right hand, and the other on the left, in thy kingdom."],
['Matthew','Matt','20','022','22',"But Jesus answered and said, Ye know not what ye ask. Are ye able to drink of the cup that I shall drink of, and to be baptized with the baptism that I am baptized with? They say unto him, We are able."],
['Matthew','Matt','20','023','23',"And he saith unto them, Ye shall drink indeed of my cup, and be baptized with the baptism that I am baptized with: but to sit on my right hand, and on my left, is not mine to give, but \emph{it} \emph{shall} \emph{be} \emph{given} \emph{to} \emph{them} for whom it is prepared of my Father."],
['Matthew','Matt','20','024','24',"And when the ten heard \emph{it}, they were moved with indignation against the two brethren."],
['Matthew','Matt','20','025','25',"But Jesus called them \emph{unto} \emph{him}, and said, Ye know that the princes of the Gentiles exercise dominion over them, and they that are great exercise authority upon them."],
['Matthew','Matt','20','026','26',"But it shall not be so among you: but whosoever will be great among you, let him be your minister;"],
['Matthew','Matt','20','027','27',"And whosoever will be chief among you, let him be your servant:"],
['Matthew','Matt','20','028','28',"Even as the Son of man came not to be ministered unto, but to minister, and to give his life a ransom for many."],
['Matthew','Matt','20','029','29',"And as they departed from Jericho, a great multitude followed him."],
['Matthew','Matt','20','030','30',"And, behold, two blind men sitting by the way side, when they heard that Jesus passed by, cried out, saying, Have mercy on us, O Lord, \emph{thou} Son of David."],
['Matthew','Matt','20','031','31',"And the multitude rebuked them, because they should hold their peace: but they cried the more, saying, Have mercy on us, O Lord, \emph{thou} Son of David."],
['Matthew','Matt','20','032','32',"And Jesus stood still, and called them, and said, What will ye that I shall do unto you?"],
['Matthew','Matt','20','033','33',"They say unto him, Lord, that our eyes may be opened."],
['Matthew','Matt','20','034','34',"So Jesus had compassion \emph{on} \emph{them}, and touched their eyes: and immediately their eyes received sight, and they followed him."]
]

Matthew21 = [
             ['Matthew','Matt','21','001','1',"And when they drew nigh unto Jerusalem, and were come to Bethphage, unto the mount of Olives, then sent Jesus two disciples,"],
['Matthew','Matt','21','002','2',"Saying unto them, Go into the village over against you, and straightway ye shall find an ass tied, and a colt with her: loose \emph{them}, and bring \emph{them} unto me."],
['Matthew','Matt','21','003','3',"And if any \emph{man} say ought unto you, ye shall say, The Lord hath need of them; and straightway he will send them."],
['Matthew','Matt','21','004','4',"All this was done, that it might be fulfilled which was spoken by the prophet, saying,"],
['Matthew','Matt','21','005','5',"Tell ye the daughter of Sion, Behold, thy King cometh unto thee, meek, and sitting upon an ass, and a colt the foal of an ass."],
['Matthew','Matt','21','006','6',"And the disciples went, and did as Jesus commanded them,"],
['Matthew','Matt','21','007','7',"And brought the ass, and the colt, and put on them their clothes, and they set \emph{him} thereon."],
['Matthew','Matt','21','008','8',"And a very great multitude spread their garments in the way; others cut down branches from the trees, and strawed \emph{them} in the way."],
['Matthew','Matt','21','009','9',"And the multitudes that went before, and that followed, cried, saying, Hosanna to the Son of David: Blessed \emph{is} he that cometh in the name of the Lord; Hosanna in the highest."],
['Matthew','Matt','21','010','10',"And when he was come into Jerusalem, all the city was moved, saying, Who is this?"],
['Matthew','Matt','21','011','11',"And the multitude said, This is Jesus the prophet of Nazareth of Galilee."],
['Matthew','Matt','21','012','12',"And Jesus went into the temple of God, and cast out all them that sold and bought in the temple, and overthrew the tables of the moneychangers, and the seats of them that sold doves,"],
['Matthew','Matt','21','013','13',"And said unto them, It is written, My house shall be called the house of prayer; but ye have made it a den of thieves."],
['Matthew','Matt','21','014','14',"And the blind and the lame came to him in the temple; and he healed them."],
['Matthew','Matt','21','015','15',"And when the chief priests and scribes saw the wonderful things that he did, and the children crying in the temple, and saying, Hosanna to the Son of David; they were sore displeased,"],
['Matthew','Matt','21','016','16',"And said unto him, Hearest thou what these say? And Jesus saith unto them, Yea; have ye never read, Out of the mouth of babes and sucklings thou hast perfected praise?"],
['Matthew','Matt','21','017','17',"And he left them, and went out of the city into Bethany; and he lodged there."],
['Matthew','Matt','21','018','18',"Now in the morning as he returned into the city, he hungered."],
['Matthew','Matt','21','019','19',"And when he saw a fig tree in the way, he came to it, and found nothing thereon, but leaves only, and said unto it, Let no fruit grow on thee henceforward for ever. And presently the fig tree withered away."],
['Matthew','Matt','21','020','20',"And when the disciples saw \emph{it}, they marvelled, saying, How soon is the fig tree withered away!"],
['Matthew','Matt','21','021','21',"Jesus answered and said unto them, Verily I say unto you, If ye have faith, and doubt not, ye shall not only do this \emph{which} \emph{is} \emph{done} to the fig tree, but also if ye shall say unto this mountain, Be thou removed, and be thou cast into the sea; it shall be done."],
['Matthew','Matt','21','022','22',"And all things, whatsoever ye shall ask in prayer, believing, ye shall receive."],
['Matthew','Matt','21','023','23',"And when he was come into the temple, the chief priests and the elders of the people came unto him as he was teaching, and said, By what authority doest thou these things? and who gave thee this authority?"],
['Matthew','Matt','21','024','24',"And Jesus answered and said unto them, I also will ask you one thing, which if ye tell me, I in like wise will tell you by what authority I do these things."],
['Matthew','Matt','21','025','25',"The baptism of John, whence was it? from heaven, or of men? And they reasoned with themselves, saying, If we shall say, From heaven; he will say unto us, Why did ye not then believe him?"],
['Matthew','Matt','21','026','26',"But if we shall say, Of men; we fear the people; for all hold John as a prophet."],
['Matthew','Matt','21','027','27',"And they answered Jesus, and said, We cannot tell. And he said unto them, Neither tell I you by what authority I do these things."],
['Matthew','Matt','21','028','28',"But what think ye? A \emph{certain} man had two sons; and he came to the first, and said, Son, go work to day in my vineyard."],
['Matthew','Matt','21','029','29',"He answered and said, I will not: but afterward he repented, and went."],
['Matthew','Matt','21','030','30',"And he came to the second, and said likewise. And he answered and said, I \emph{go}, sir: and went not."],
['Matthew','Matt','21','031','31',"Whether of them twain did the will of \emph{his} father? They say unto him, The first. Jesus saith unto them, Verily I say unto you, That the publicans and the harlots go into the kingdom of God before you."],
['Matthew','Matt','21','032','32',"For John came unto you in the way of righteousness, and ye believed him not: but the publicans and the harlots believed him: and ye, when ye had seen \emph{it}, repented not afterward, that ye might believe him."],
['Matthew','Matt','21','033','33',"Hear another parable: There was a certain householder, which planted a vineyard, and hedged it round about, and digged a winepress in it, and built a tower, and let it out to husbandmen, and went into a far country:"],
['Matthew','Matt','21','034','34',"And when the time of the fruit drew near, he sent his servants to the husbandmen, that they might receive the fruits of it."],
['Matthew','Matt','21','035','35',"And the husbandmen took his servants, and beat one, and killed another, and stoned another."],
['Matthew','Matt','21','036','36',"Again, he sent other servants more than the first: and they did unto them likewise."],
['Matthew','Matt','21','037','37',"But last of all he sent unto them his son, saying, They will reverence my son."],
['Matthew','Matt','21','038','38',"But when the husbandmen saw the son, they said among themselves, This is the heir; come, let us kill him, and let us seize on his inheritance."],
['Matthew','Matt','21','039','39',"And they caught him, and cast \emph{him} out of the vineyard, and slew \emph{him}."],
['Matthew','Matt','21','040','40',"When the lord therefore of the vineyard cometh, what will he do unto those husbandmen?"],
['Matthew','Matt','21','041','41',"They say unto him, He will miserably destroy those wicked men, and will let out \emph{his} vineyard unto other husbandmen, which shall render him the fruits in their seasons."],
['Matthew','Matt','21','042','42',"Jesus saith unto them, Did ye never read in the scriptures, The stone which the builders rejected, the same is become the head of the corner: this is the Lord's doing, and it is marvellous in our eyes?"],
['Matthew','Matt','21','043','43',"Therefore say I unto you, The kingdom of God shall be taken from you, and given to a nation bringing forth the fruits thereof."],
['Matthew','Matt','21','044','44',"And whosoever shall fall on this stone shall be broken: but on whomsoever it shall fall, it will grind him to powder."],
['Matthew','Matt','21','045','45',"And when the chief priests and Pharisees had heard his parables, they perceived that he spake of them."],
['Matthew','Matt','21','046','46',"But when they sought to lay hands on him, they feared the multitude, because they took him for a prophet."]
]

Matthew22 = [
['Matthew','Matt','22','001','1',"And Jesus answered and spake unto them again by parables, and said,"],
['Matthew','Matt','22','002','2',"The kingdom of heaven is like unto a certain king, which made a marriage for his son,"],
['Matthew','Matt','22','003','3',"And sent forth his servants to call them that were bidden to the wedding: and they would not come."],
['Matthew','Matt','22','004','4',"Again, he sent forth other servants, saying, Tell them which are bidden, Behold, I have prepared my dinner: my oxen and \emph{my} fatlings \emph{are} killed, and all things \emph{are} ready: come unto the marriage."],
['Matthew','Matt','22','005','5',"But they made light of \emph{it}, and went their ways, one to his farm, another to his merchandise:"],
['Matthew','Matt','22','006','6',"And the remnant took his servants, and entreated \emph{them} spitefully, and slew \emph{them}."],
['Matthew','Matt','22','007','7',"But when the king heard \emph{thereof}, he was wroth: and he sent forth his armies, and destroyed those murderers, and burned up their city."],
['Matthew','Matt','22','008','8',"Then saith he to his servants, The wedding is ready, but they which were bidden were not worthy."],
['Matthew','Matt','22','009','9',"Go ye therefore into the highways, and as many as ye shall find, bid to the marriage."],
['Matthew','Matt','22','010','10',"So those servants went out into the highways, and gathered together all as many as they found, both bad and good: and the wedding was furnished with guests."],
['Matthew','Matt','22','011','11',"And when the king came in to see the guests, he saw there a man which had not on a wedding garment:"],
['Matthew','Matt','22','012','12',"And he saith unto him, Friend, how camest thou in hither not having a wedding garment? And he was speechless."],
['Matthew','Matt','22','013','13',"Then said the king to the servants, Bind him hand and foot, and take him away, and cast \emph{him} into outer darkness; there shall be weeping and gnashing of teeth."],
['Matthew','Matt','22','014','14',"For many are called, but few \emph{are} chosen."],
['Matthew','Matt','22','015','15',"Then went the Pharisees, and took counsel how they might entangle him in \emph{his} talk."],
['Matthew','Matt','22','016','16',"And they sent out unto him their disciples with the Herodians, saying, Master, we know that thou art true, and teachest the way of God in truth, neither carest thou for any \emph{man}: for thou regardest not the person of men."],
['Matthew','Matt','22','017','17',"Tell us therefore, What thinkest thou? Is it lawful to give tribute unto Caesar, or not?"],
['Matthew','Matt','22','018','18',"But Jesus perceived their wickedness, and said, Why tempt ye me, \emph{ye} hypocrites?"],
['Matthew','Matt','22','019','19',"Shew me the tribute money. And they brought unto him a penny."],
['Matthew','Matt','22','020','20',"And he saith unto them, Whose \emph{is} this image and superscription?"],
['Matthew','Matt','22','021','21',"They say unto him, Caesar's. Then saith he unto them, Render therefore unto Caesar the things which are Caesar's; and unto God the things that are God's."],
['Matthew','Matt','22','022','22',"When they had heard \emph{these} \emph{words}, they marvelled, and left him, and went their way."],
['Matthew','Matt','22','023','23',"The same day came to him the Sadducees, which say that there is no resurrection, and asked him,"],
['Matthew','Matt','22','024','24',"Saying, Master, Moses said, If a man die, having no children, his brother shall marry his wife, and raise up seed unto his brother."],
['Matthew','Matt','22','025','25',"Now there were with us seven brethren: and the first, when he had married a wife, deceased, and, having no issue, left his wife unto his brother:"],
['Matthew','Matt','22','026','26',"Likewise the second also, and the third, unto the seventh."],
['Matthew','Matt','22','027','27',"And last of all the woman died also."],
['Matthew','Matt','22','028','28',"Therefore in the resurrection whose wife shall she be of the seven? for they all had her."],
['Matthew','Matt','22','029','29',"Jesus answered and said unto them, Ye do err, not knowing the scriptures, nor the power of God."],
['Matthew','Matt','22','030','30',"For in the resurrection they neither marry, nor are given in marriage, but are as the angels of God in heaven."],
['Matthew','Matt','22','031','31',"But as touching the resurrection of the dead, have ye not read that which was spoken unto you by God, saying,"],
['Matthew','Matt','22','032','32',"I am the God of Abraham, and the God of Isaac, and the God of Jacob? God is not the God of the dead, but of the living."],
['Matthew','Matt','22','033','33',"And when the multitude heard \emph{this}, they were astonished at his doctrine."],
['Matthew','Matt','22','034','34',"But when the Pharisees had heard that he had put the Sadducees to silence, they were gathered together."],
['Matthew','Matt','22','035','35',"Then one of them, \emph{which} \emph{was} a lawyer, asked \emph{him} \emph{a} \emph{question}, tempting him, and saying,"],
['Matthew','Matt','22','036','36',"Master, which \emph{is} the great commandment in the law?"],
['Matthew','Matt','22','037','37',"Jesus said unto him, Thou shalt love the Lord thy God with all thy heart, and with all thy soul, and with all thy mind."],
['Matthew','Matt','22','038','38',"This is the first and great commandment."],
['Matthew','Matt','22','039','39',"And the second \emph{is} like unto it, Thou shalt love thy neighbour as thyself."],
['Matthew','Matt','22','040','40',"On these two commandments hang all the law and the prophets."],
['Matthew','Matt','22','041','41',"While the Pharisees were gathered together, Jesus asked them,"],
['Matthew','Matt','22','042','42',"Saying, What think ye of Christ? whose son is he? They say unto him, \emph{The} \emph{Son} of David."],
['Matthew','Matt','22','043','43',"He saith unto them, How then doth David in spirit call him Lord, saying,"],
['Matthew','Matt','22','044','44',"The LORD said unto my Lord, Sit thou on my right hand, till I make thine enemies thy footstool?"],
['Matthew','Matt','22','045','45',"If David then call him Lord, how is he his son?"],
['Matthew','Matt','22','046','46',"And no man was able to answer him a word, neither durst any \emph{man} from that day forth ask him any more \emph{questions}."]
]

Matthew23 = [
             ['Matthew','Matt','23','001','1',"Then spake Jesus to the multitude, and to his disciples,"],
['Matthew','Matt','23','002','2',"Saying, The scribes and the Pharisees sit in Moses' seat:"],
['Matthew','Matt','23','003','3',"All therefore whatsoever they bid you observe, \emph{that} observe and do; but do not ye after their works: for they say, and do not."],
['Matthew','Matt','23','004','4',"For they bind heavy burdens and grievous to be borne, and lay \emph{them} on men's shoulders; but they \emph{themselves} will not move them with one of their fingers."],
['Matthew','Matt','23','005','5',"But all their works they do for to be seen of men: they make broad their phylacteries, and enlarge the borders of their garments,"],
['Matthew','Matt','23','006','6',"And love the uppermost rooms at feasts, and the chief seats in the synagogues,"],
['Matthew','Matt','23','007','7',"And greetings in the markets, and to be called of men, Rabbi, Rabbi."],
['Matthew','Matt','23','008','8',"But be not ye called Rabbi: for one is your Master, \emph{even} Christ; and all ye are brethren."],
['Matthew','Matt','23','009','9',"And call no \emph{man} your father upon the earth: for one is your Father, which is in heaven."],
['Matthew','Matt','23','010','10',"Neither be ye called masters: for one is your Master, \emph{even} Christ."],
['Matthew','Matt','23','011','11',"But he that is greatest among you shall be your servant."],
['Matthew','Matt','23','012','12',"And whosoever shall exalt himself shall be abased; and he that shall humble himself shall be exalted."],
['Matthew','Matt','23','013','13',"But woe unto you, scribes and Pharisees, hypocrites! for ye shut up the kingdom of heaven against men: for ye neither go in \emph{yourselves}, neither suffer ye them that are entering to go in."],
['Matthew','Matt','23','014','14',"Woe unto you, scribes and Pharisees, hypocrites! for ye devour widows' houses, and for a pretence make long prayer: therefore ye shall receive the greater damnation."],
['Matthew','Matt','23','015','15',"Woe unto you, scribes and Pharisees, hypocrites! for ye compass sea and land to make one proselyte, and when he is made, ye make him twofold more the child of hell than yourselves."],
['Matthew','Matt','23','016','16',"Woe unto you, \emph{ye} blind guides, which say, Whosoever shall swear by the temple, it is nothing; but whosoever shall swear by the gold of the temple, he is a debtor!"],
['Matthew','Matt','23','017','17',"\emph{Ye} fools and blind: for whether is greater, the gold, or the temple that sanctifieth the gold?"],
['Matthew','Matt','23','018','18',"And, Whosoever shall swear by the altar, it is nothing; but whosoever sweareth by the gift that is upon it, he is guilty."],
['Matthew','Matt','23','019','19',"\emph{Ye} fools and blind: for whether \emph{is} greater, the gift, or the altar that sanctifieth the gift?"],
['Matthew','Matt','23','020','20',"Whoso therefore shall swear by the altar, sweareth by it, and by all things thereon."],
['Matthew','Matt','23','021','21',"And whoso shall swear by the temple, sweareth by it, and by him that dwelleth therein."],
['Matthew','Matt','23','022','22',"And he that shall swear by heaven, sweareth by the throne of God, and by him that sitteth thereon."],
['Matthew','Matt','23','023','23',"Woe unto you, scribes and Pharisees, hypocrites! for ye pay tithe of mint and anise and cummin, and have omitted the weightier \emph{matters} of the law, judgment, mercy, and faith: these ought ye to have done, and not to leave the other undone."],
['Matthew','Matt','23','024','24',"\emph{Ye} blind guides, which strain at a gnat, and swallow a camel."],
['Matthew','Matt','23','025','25',"Woe unto you, scribes and Pharisees, hypocrites! for ye make clean the outside of the cup and of the platter, but within they are full of extortion and excess."],
['Matthew','Matt','23','026','26',"\emph{Thou} blind Pharisee, cleanse first that \emph{which} \emph{is} within the cup and platter, that the outside of them may be clean also."],
['Matthew','Matt','23','027','27',"Woe unto you, scribes and Pharisees, hypocrites! for ye are like unto whited sepulchres, which indeed appear beautiful outward, but are within full of dead \emph{men's} bones, and of all uncleanness."],
['Matthew','Matt','23','028','28',"Even so ye also outwardly appear righteous unto men, but within ye are full of hypocrisy and iniquity."],
['Matthew','Matt','23','029','29',"Woe unto you, scribes and Pharisees, hypocrites! because ye build the tombs of the prophets, and garnish the sepulchres of the righteous,"],
['Matthew','Matt','23','030','30',"And say, If we had been in the days of our fathers, we would not have been partakers with them in the blood of the prophets."],
['Matthew','Matt','23','031','31',"Wherefore ye be witnesses unto yourselves, that ye are the children of them which killed the prophets."],
['Matthew','Matt','23','032','32',"Fill ye up then the measure of your fathers."],
['Matthew','Matt','23','033','33',"\emph{Ye} serpents, \emph{ye} generation of vipers, how can ye escape the damnation of hell?"],
['Matthew','Matt','23','034','34',"Wherefore, behold, I send unto you prophets, and wise men, and scribes: and \emph{some} of them ye shall kill and crucify; and \emph{some} of them shall ye scourge in your synagogues, and persecute \emph{them} from city to city:"],
['Matthew','Matt','23','035','35',"That upon you may come all the righteous blood shed upon the earth, from the blood of righteous Abel unto the blood of Zacharias son of Barachias, whom ye slew between the temple and the altar."],
['Matthew','Matt','23','036','36',"Verily I say unto you, All these things shall come upon this generation."],
['Matthew','Matt','23','037','37',"O Jerusalem, Jerusalem, \emph{thou} that killest the prophets, and stonest them which are sent unto thee, how often would I have gathered thy children together, even as a hen gathereth her chickens under \emph{her} wings, and ye would not!"],
['Matthew','Matt','23','038','38',"Behold, your house is left unto you desolate."],
['Matthew','Matt','23','039','39',"For I say unto you, Ye shall not see me henceforth, till ye shall say, Blessed \emph{is} he that cometh in the name of the Lord."]
]

Matthew24 = [
        ['Matthew','Matt','24','001','1',"And Jesus went out, and departed from the temple: and his disciples came to \emph{him} for to shew him the buildings of the temple."],
['Matthew','Matt','24','002','2',"And Jesus said unto them, See ye not all these things? verily I say unto you, There shall not be left here one stone upon another, that shall not be thrown down."],
['Matthew','Matt','24','003','3',"And as he sat upon the mount of Olives, the disciples came unto him privately, saying, Tell us, when shall these things be? and what \emph{shall} \emph{be} the sign of thy coming, and of the end of the world?"],
['Matthew','Matt','24','004','4',"And Jesus answered and said unto them, Take heed that no man deceive you."],
['Matthew','Matt','24','005','5',"For many shall come in my name, saying, I am Christ; and shall deceive many."],
['Matthew','Matt','24','006','6',"And ye shall hear of wars and rumours of wars: see that ye be not troubled: for all \emph{these} \emph{things} must come to pass, but the end is not yet."],
['Matthew','Matt','24','007','7',"For nation shall rise against nation, and kingdom against kingdom: and there shall be famines, and pestilences, and earthquakes, in divers places."],
['Matthew','Matt','24','008','8',"All these \emph{are} the beginning of sorrows."],
['Matthew','Matt','24','009','9',"Then shall they deliver you up to be afflicted, and shall kill you: and ye shall be hated of all nations for my name's sake."],
['Matthew','Matt','24','010','10',"And then shall many be offended, and shall betray one another, and shall hate one another."],
['Matthew','Matt','24','011','11',"And many false prophets shall rise, and shall deceive many."],
['Matthew','Matt','24','012','12',"And because iniquity shall abound, the love of many shall wax cold."],
['Matthew','Matt','24','013','13',"But he that shall endure unto the end, the same shall be saved."],
['Matthew','Matt','24','014','14',"And this gospel of the kingdom shall be preached in all the world for a witness unto all nations; and then shall the end come."],
['Matthew','Matt','24','015','15',"When ye therefore shall see the abomination of desolation, spoken of by Daniel the prophet, stand in the holy place, (whoso readeth, let him understand:)"],
['Matthew','Matt','24','016','16',"Then let them which be in Judaea flee into the mountains:"],
['Matthew','Matt','24','017','17',"Let him which is on the housetop not come down to take any thing out of his house:"],
['Matthew','Matt','24','018','18',"Neither let him which is in the field return back to take his clothes."],
['Matthew','Matt','24','019','19',"And woe unto them that are with child, and to them that give suck in those days!"],
['Matthew','Matt','24','020','20',"But pray ye that your flight be not in the winter, neither on the sabbath day:"],
['Matthew','Matt','24','021','21',"For then shall be great tribulation, such as was not since the beginning of the world to this time, no, nor ever shall be."],
['Matthew','Matt','24','022','22',"And except those days should be shortened, there should no flesh be saved: but for the elect's sake those days shall be shortened."],
['Matthew','Matt','24','023','23',"Then if any man shall say unto you, Lo, here \emph{is} Christ, or there; believe \emph{it} not."],
['Matthew','Matt','24','024','24',"For there shall arise false Christs, and false prophets, and shall shew great signs and wonders; insomuch that, if \emph{it} \emph{were} possible, they shall deceive the very elect."],
['Matthew','Matt','24','025','25',"Behold, I have told you before."],
['Matthew','Matt','24','026','26',"Wherefore if they shall say unto you, Behold, he is in the desert; go not forth: behold, \emph{he} \emph{is} in the secret chambers; believe \emph{it} not."],
['Matthew','Matt','24','027','27',"For as the lightning cometh out of the east, and shineth even unto the west; so shall also the coming of the Son of man be."],
['Matthew','Matt','24','028','28',"For wheresoever the carcase is, there will the eagles be gathered together."],
['Matthew','Matt','24','029','29',"Immediately after the tribulation of those days shall the sun be darkened, and the moon shall not give her light, and the stars shall fall from heaven, and the powers of the heavens shall be shaken:"],
['Matthew','Matt','24','030','30',"And then shall appear the sign of the Son of man in heaven: and then shall all the tribes of the earth mourn, and they shall see the Son of man coming in the clouds of heaven with power and great glory."],
['Matthew','Matt','24','031','31',"And he shall send his angels with a great sound of a trumpet, and they shall gather together his elect from the four winds, from one end of heaven to the other."],
['Matthew','Matt','24','032','32',"Now learn a parable of the fig tree; When his branch is yet tender, and putteth forth leaves, ye know that summer \emph{is} nigh:"],
['Matthew','Matt','24','033','33',"So likewise ye, when ye shall see all these things, know that it is near, \emph{even} at the doors."],
['Matthew','Matt','24','034','34',"Verily I say unto you, This generation shall not pass, till all these things be fulfilled."],
['Matthew','Matt','24','035','35',"Heaven and earth shall pass away, but my words shall not pass away."],
['Matthew','Matt','24','036','36',"But of that day and hour knoweth no \emph{man}, no, not the angels of heaven, but my Father only."],
['Matthew','Matt','24','037','37',"But as the days of Noe \emph{were}, so shall also the coming of the Son of man be."],
['Matthew','Matt','24','038','38',"For as in the days that were before the flood they were eating and drinking, marrying and giving in marriage, until the day that Noe entered into the ark,"],
['Matthew','Matt','24','039','39',"And knew not until the flood came, and took them all away; so shall also the coming of the Son of man be."],
['Matthew','Matt','24','040','40',"Then shall two be in the field; the one shall be taken, and the other left."],
['Matthew','Matt','24','041','41',"Two \emph{women} \emph{shall} \emph{be} grinding at the mill; the one shall be taken, and the other left."],
['Matthew','Matt','24','042','42',"Watch therefore: for ye know not what hour your Lord doth come."],
['Matthew','Matt','24','043','43',"But know this, that if the goodman of the house had known in what watch the thief would come, he would have watched, and would not have suffered his house to be broken up."],
['Matthew','Matt','24','044','44',"Therefore be ye also ready: for in such an hour as ye think not the Son of man cometh."],
['Matthew','Matt','24','045','45',"Who then is a faithful and wise servant, whom his lord hath made ruler over his household, to give them meat in due season?"],
['Matthew','Matt','24','046','46',"Blessed \emph{is} that servant, whom his lord when he cometh shall find so doing."],
['Matthew','Matt','24','047','47',"Verily I say unto you, That he shall make him ruler over all his goods."],
['Matthew','Matt','24','048','48',"But and if that evil servant shall say in his heart, My lord delayeth his coming;"],
['Matthew','Matt','24','049','49',"And shall begin to smite \emph{his} fellowservants, and to eat and drink with the drunken;"],
['Matthew','Matt','24','050','50',"The lord of that servant shall come in a day when he looketh not for \emph{him}, and in an hour that he is not aware of,"],
['Matthew','Matt','24','051','51',"And shall cut him asunder, and appoint \emph{him} his portion with the hypocrites: there shall be weeping and gnashing of teeth."]
]

Matthew25 = [
             ['Matthew','Matt','25','001','1',"Then shall the kingdom of heaven be likened unto ten virgins, which took their lamps, and went forth to meet the bridegroom."],
['Matthew','Matt','25','002','2',"And five of them were wise, and five \emph{were} foolish."],
['Matthew','Matt','25','003','3',"They that \emph{were} foolish took their lamps, and took no oil with them:"],
['Matthew','Matt','25','004','4',"But the wise took oil in their vessels with their lamps."],
['Matthew','Matt','25','005','5',"While the bridegroom tarried, they all slumbered and slept."],
['Matthew','Matt','25','006','6',"And at midnight there was a cry made, Behold, the bridegroom cometh; go ye out to meet him."],
['Matthew','Matt','25','007','7',"Then all those virgins arose, and trimmed their lamps."],
['Matthew','Matt','25','008','8',"And the foolish said unto the wise, Give us of your oil; for our lamps are gone out."],
['Matthew','Matt','25','009','9',"But the wise answered, saying, \emph{Not} \emph{so}; lest there be not enough for us and you: but go ye rather to them that sell, and buy for yourselves."],
['Matthew','Matt','25','010','10',"And while they went to buy, the bridegroom came; and they that were ready went in with him to the marriage: and the door was shut."],
['Matthew','Matt','25','011','11',"Afterward came also the other virgins, saying, Lord, Lord, open to us."],
['Matthew','Matt','25','012','12',"But he answered and said, Verily I say unto you, I know you not."],
['Matthew','Matt','25','013','13',"Watch therefore, for ye know neither the day nor the hour wherein the Son of man cometh."],
['Matthew','Matt','25','014','14',"For \emph{the} \emph{kingdom} \emph{of} \emph{heaven} \emph{is} as a man travelling into a far country, \emph{who} called his own servants, and delivered unto them his goods."],
['Matthew','Matt','25','015','15',"And unto one he gave five talents, to another two, and to another one; to every man according to his several ability; and straightway took his journey."],
['Matthew','Matt','25','016','16',"Then he that had received the five talents went and traded with the same, and made \emph{them} other five talents."],
['Matthew','Matt','25','017','17',"And likewise he that \emph{had} \emph{received} two, he also gained other two."],
['Matthew','Matt','25','018','18',"But he that had received one went and digged in the earth, and hid his lord's money."],
['Matthew','Matt','25','019','19',"After a long time the lord of those servants cometh, and reckoneth with them."],
['Matthew','Matt','25','020','20',"And so he that had received five talents came and brought other five talents, saying, Lord, thou deliveredst unto me five talents: behold, I have gained beside them five talents more."],
['Matthew','Matt','25','021','21',"His lord said unto him, Well done, \emph{thou} good and faithful servant: thou hast been faithful over a few things, I will make thee ruler over many things: enter thou into the joy of thy lord."],
['Matthew','Matt','25','022','22',"He also that had received two talents came and said, Lord, thou deliveredst unto me two talents: behold, I have gained two other talents beside them."],
['Matthew','Matt','25','023','23',"His lord said unto him, Well done, good and faithful servant; thou hast been faithful over a few things, I will make thee ruler over many things: enter thou into the joy of thy lord."],
['Matthew','Matt','25','024','24',"Then he which had received the one talent came and said, Lord, I knew thee that thou art an hard man, reaping where thou hast not sown, and gathering where thou hast not strawed:"],
['Matthew','Matt','25','025','25',"And I was afraid, and went and hid thy talent in the earth: lo, \emph{there} thou hast \emph{that} \emph{is} thine."],
['Matthew','Matt','25','026','26',"His lord answered and said unto him, \emph{Thou} wicked and slothful servant, thou knewest that I reap where I sowed not, and gather where I have not strawed:"],
['Matthew','Matt','25','027','27',"Thou oughtest therefore to have put my money to the exchangers, and \emph{then} at my coming I should have received mine own with usury."],
['Matthew','Matt','25','028','28',"Take therefore the talent from him, and give \emph{it} unto him which hath ten talents."],
['Matthew','Matt','25','029','29',"For unto every one that hath shall be given, and he shall have abundance: but from him that hath not shall be taken away even that which he hath."],
['Matthew','Matt','25','030','30',"And cast ye the unprofitable servant into outer darkness: there shall be weeping and gnashing of teeth."],
['Matthew','Matt','25','031','31',"When the Son of man shall come in his glory, and all the holy angels with him, then shall he sit upon the throne of his glory:"],
['Matthew','Matt','25','032','32',"And before him shall be gathered all nations: and he shall separate them one from another, as a shepherd divideth \emph{his} sheep from the goats:"],
['Matthew','Matt','25','033','33',"And he shall set the sheep on his right hand, but the goats on the left."],
['Matthew','Matt','25','034','34',"Then shall the King say unto them on his right hand, Come, ye blessed of my Father, inherit the kingdom prepared for you from the foundation of the world:"],
['Matthew','Matt','25','035','35',"For I was an hungred, and ye gave me meat: I was thirsty, and ye gave me drink: I was a stranger, and ye took me in:"],
['Matthew','Matt','25','036','36',"Naked, and ye clothed me: I was sick, and ye visited me: I was in prison, and ye came unto me."],
['Matthew','Matt','25','037','37',"Then shall the righteous answer him, saying, Lord, when saw we thee an hungred, and fed \emph{thee}? or thirsty, and gave \emph{thee} drink?"],
['Matthew','Matt','25','038','38',"When saw we thee a stranger, and took \emph{thee} in? or naked, and clothed \emph{thee}?"],
['Matthew','Matt','25','039','39',"Or when saw we thee sick, or in prison, and came unto thee?"],
['Matthew','Matt','25','040','40',"And the King shall answer and say unto them, Verily I say unto you, Inasmuch as ye have done \emph{it} unto one of the least of these my brethren, ye have done \emph{it} unto me."],
['Matthew','Matt','25','041','41',"Then shall he say also unto them on the left hand, Depart from me, ye cursed, into everlasting fire, prepared for the devil and his angels:"],
['Matthew','Matt','25','042','42',"For I was an hungred, and ye gave me no meat: I was thirsty, and ye gave me no drink:"],
['Matthew','Matt','25','043','43',"I was a stranger, and ye took me not in: naked, and ye clothed me not: sick, and in prison, and ye visited me not."],
['Matthew','Matt','25','044','44',"Then shall they also answer him, saying, Lord, when saw we thee an hungred, or athirst, or a stranger, or naked, or sick, or in prison, and did not minister unto thee?"],
['Matthew','Matt','25','045','45',"Then shall he answer them, saying, Verily I say unto you, Inasmuch as ye did \emph{it} not to one of the least of these, ye did \emph{it} not to me."],
['Matthew','Matt','25','046','46',"And these shall go away into everlasting punishment: but the righteous into life eternal."]
]

Matthew26 = [
             ['Matthew','Matt','26','001','1',"And it came to pass, when Jesus had finished all these sayings, he said unto his disciples,"],
['Matthew','Matt','26','002','2',"Ye know that after two days is \emph{the} \emph{feast} \emph{of} the passover, and the Son of man is betrayed to be crucified."],
['Matthew','Matt','26','003','3',"Then assembled together the chief priests, and the scribes, and the elders of the people, unto the palace of the high priest, who was called Caiaphas,"],
['Matthew','Matt','26','004','4',"And consulted that they might take Jesus by subtilty, and kill \emph{him}."],
['Matthew','Matt','26','005','5',"But they said, Not on the feast \emph{day}, lest there be an uproar among the people."],
['Matthew','Matt','26','006','6',"Now when Jesus was in Bethany, in the house of Simon the leper,"],
['Matthew','Matt','26','007','7',"There came unto him a woman having an alabaster box of very precious ointment, and poured it on his head, as he sat \emph{at} \emph{meat}."],
['Matthew','Matt','26','008','8',"But when his disciples saw \emph{it}, they had indignation, saying, To what purpose \emph{is} this waste?"],
['Matthew','Matt','26','009','9',"For this ointment might have been sold for much, and given to the poor."],
['Matthew','Matt','26','010','10',"When Jesus understood \emph{it}, he said unto them, Why trouble ye the woman? for she hath wrought a good work upon me."],
['Matthew','Matt','26','011','11',"For ye have the poor always with you; but me ye have not always."],
['Matthew','Matt','26','012','12',"For in that she hath poured this ointment on my body, she did \emph{it} for my burial."],
['Matthew','Matt','26','013','13',"Verily I say unto you, Wheresoever this gospel shall be preached in the whole world, \emph{there} shall also this, that this woman hath done, be told for a memorial of her."],
['Matthew','Matt','26','014','14',"Then one of the twelve, called Judas Iscariot, went unto the chief priests,"],
['Matthew','Matt','26','015','15',"And said \emph{unto} \emph{them}, What will ye give me, and I will deliver him unto you? And they covenanted with him for thirty pieces of silver."],
['Matthew','Matt','26','016','16',"And from that time he sought opportunity to betray him."],
['Matthew','Matt','26','017','17',"Now the first \emph{day} of the \emph{feast} \emph{of} unleavened bread the disciples came to Jesus, saying unto him, Where wilt thou that we prepare for thee to eat the passover?"],
['Matthew','Matt','26','018','18',"And he said, Go into the city to such a man, and say unto him, The Master saith, My time is at hand; I will keep the passover at thy house with my disciples."],
['Matthew','Matt','26','019','19',"And the disciples did as Jesus had appointed them; and they made ready the passover."],
['Matthew','Matt','26','020','20',"Now when the even was come, he sat down with the twelve."],
['Matthew','Matt','26','021','21',"And as they did eat, he said, Verily I say unto you, that one of you shall betray me."],
['Matthew','Matt','26','022','22',"And they were exceeding sorrowful, and began every one of them to say unto him, Lord, is it I?"],
['Matthew','Matt','26','023','23',"And he answered and said, He that dippeth \emph{his} hand with me in the dish, the same shall betray me."],
['Matthew','Matt','26','024','24',"The Son of man goeth as it is written of him: but woe unto that man by whom the Son of man is betrayed! it had been good for that man if he had not been born."],
['Matthew','Matt','26','025','25',"Then Judas, which betrayed him, answered and said, Master, is it I? He said unto him, Thou hast said."],
['Matthew','Matt','26','026','26',"And as they were eating, Jesus took bread, and blessed \emph{it}, and brake \emph{it}, and gave \emph{it} to the disciples, and said, Take, eat; this is my body."],
['Matthew','Matt','26','027','27',"And he took the cup, and gave thanks, and gave \emph{it} to them, saying, Drink ye all of it;"],
['Matthew','Matt','26','028','28',"For this is my blood of the new testament, which is shed for many for the remission of sins."],
['Matthew','Matt','26','029','29',"But I say unto you, I will not drink henceforth of this fruit of the vine, until that day when I drink it new with you in my Father's kingdom."],
['Matthew','Matt','26','030','30',"And when they had sung an hymn, they went out into the mount of Olives."],
['Matthew','Matt','26','031','31',"Then saith Jesus unto them, All ye shall be offended because of me this night: for it is written, I will smite the shepherd, and the sheep of the flock shall be scattered abroad."],
['Matthew','Matt','26','032','32',"But after I am risen again, I will go before you into Galilee."],
['Matthew','Matt','26','033','33',"Peter answered and said unto him, Though all \emph{men} shall be offended because of thee, \emph{yet} will I never be offended."],
['Matthew','Matt','26','034','34',"Jesus said unto him, Verily I say unto thee, That this night, before the cock crow, thou shalt deny me thrice."],
['Matthew','Matt','26','035','35',"Peter said unto him, Though I should die with thee, yet will I not deny thee. Likewise also said all the disciples."],
['Matthew','Matt','26','036','36',"Then cometh Jesus with them unto a place called Gethsemane, and saith unto the disciples, Sit ye here, while I go and pray yonder."],
['Matthew','Matt','26','037','37',"And he took with him Peter and the two sons of Zebedee, and began to be sorrowful and very heavy."],
['Matthew','Matt','26','038','38',"Then saith he unto them, My soul is exceeding sorrowful, even unto death: tarry ye here, and watch with me."],
['Matthew','Matt','26','039','39',"And he went a little further, and fell on his face, and prayed, saying, O my Father, if it be possible, let this cup pass from me: nevertheless not as I will, but as thou \emph{wilt}."],
['Matthew','Matt','26','040','40',"And he cometh unto the disciples, and findeth them asleep, and saith unto Peter, What, could ye not watch with me one hour?"],
['Matthew','Matt','26','041','41',"Watch and pray, that ye enter not into temptation: the spirit indeed \emph{is} willing, but the flesh \emph{is} weak."],
['Matthew','Matt','26','042','42',"He went away again the second time, and prayed, saying, O my Father, if this cup may not pass away from me, except I drink it, thy will be done."],
['Matthew','Matt','26','043','43',"And he came and found them asleep again: for their eyes were heavy."],
['Matthew','Matt','26','044','44',"And he left them, and went away again, and prayed the third time, saying the same words."],
['Matthew','Matt','26','045','45',"Then cometh he to his disciples, and saith unto them, Sleep on now, and take \emph{your} rest: behold, the hour is at hand, and the Son of man is betrayed into the hands of sinners."],
['Matthew','Matt','26','046','46',"Rise, let us be going: behold, he is at hand that doth betray me."],
['Matthew','Matt','26','047','47',"And while he yet spake, lo, Judas, one of the twelve, came, and with him a great multitude with swords and staves, from the chief priests and elders of the people."],
['Matthew','Matt','26','048','48',"Now he that betrayed him gave them a sign, saying, Whomsoever I shall kiss, that same is he: hold him fast."],
['Matthew','Matt','26','049','49',"And forthwith he came to Jesus, and said, Hail, master; and kissed him."],
['Matthew','Matt','26','050','50',"And Jesus said unto him, Friend, wherefore art thou come? Then came they, and laid hands on Jesus, and took him."],
['Matthew','Matt','26','051','51',"And, behold, one of them which were with Jesus stretched out \emph{his} hand, and drew his sword, and struck a servant of the high priest's, and smote off his ear."],
['Matthew','Matt','26','052','52',"Then said Jesus unto him, Put up again thy sword into his place: for all they that take the sword shall perish with the sword."],
['Matthew','Matt','26','053','53',"Thinkest thou that I cannot now pray to my Father, and he shall presently give me more than twelve legions of angels?"],
['Matthew','Matt','26','054','54',"But how then shall the scriptures be fulfilled, that thus it must be?"],
['Matthew','Matt','26','055','55',"In that same hour said Jesus to the multitudes, Are ye come out as against a thief with swords and staves for to take me? I sat daily with you teaching in the temple, and ye laid no hold on me."],
['Matthew','Matt','26','056','56',"But all this was done, that the scriptures of the prophets might be fulfilled. Then all the disciples forsook him, and fled."],
['Matthew','Matt','26','057','57',"And they that had laid hold on Jesus led \emph{him} away to Caiaphas the high priest, where the scribes and the elders were assembled."],
['Matthew','Matt','26','058','58',"But Peter followed him afar off unto the high priest's palace, and went in, and sat with the servants, to see the end."],
['Matthew','Matt','26','059','59',"Now the chief priests, and elders, and all the council, sought false witness against Jesus, to put him to death;"],
['Matthew','Matt','26','060','60',"But found none: yea, though many false witnesses came, \emph{yet} found they none. At the last came two false witnesses,"],
['Matthew','Matt','26','061','61',"And said, This \emph{fellow} said, I am able to destroy the temple of God, and to build it in three days."],
['Matthew','Matt','26','062','62',"And the high priest arose, and said unto him, Answerest thou nothing? what \emph{is} \emph{it} \emph{which} these witness against thee?"],
['Matthew','Matt','26','063','63',"But Jesus held his peace. And the high priest answered and said unto him, I adjure thee by the living God, that thou tell us whether thou be the Christ, the Son of God."],
['Matthew','Matt','26','064','64',"Jesus saith unto him, Thou hast said: nevertheless I say unto you, Hereafter shall ye see the Son of man sitting on the right hand of power, and coming in the clouds of heaven."],
['Matthew','Matt','26','065','65',"Then the high priest rent his clothes, saying, He hath spoken blasphemy; what further need have we of witnesses? behold, now ye have heard his blasphemy."],
['Matthew','Matt','26','066','66',"What think ye? They answered and said, He is guilty of death."],
['Matthew','Matt','26','067','67',"Then did they spit in his face, and buffeted him; and others smote \emph{him} with the palms of their hands,"],
['Matthew','Matt','26','068','68',"Saying, Prophesy unto us, thou Christ, Who is he that smote thee?"],
['Matthew','Matt','26','069','69',"Now Peter sat without in the palace: and a damsel came unto him, saying, Thou also wast with Jesus of Galilee."],
['Matthew','Matt','26','070','70',"But he denied before \emph{them} all, saying, I know not what thou sayest."],
['Matthew','Matt','26','071','71',"And when he was gone out into the porch, another \emph{maid} saw him, and said unto them that were there, This \emph{fellow} was also with Jesus of Nazareth."],
['Matthew','Matt','26','072','72',"And again he denied with an oath, I do not know the man."],
['Matthew','Matt','26','073','73',"And after a while came unto \emph{him} they that stood by, and said to Peter, Surely thou also art \emph{one} of them; for thy speech bewrayeth thee."],
['Matthew','Matt','26','074','74',"Then began he to curse and to swear, \emph{saying}, I know not the man. And immediately the cock crew."],
['Matthew','Matt','26','075','75',"And Peter remembered the word of Jesus, which said unto him, Before the cock crow, thou shalt deny me thrice. And he went out, and wept bitterly."]
]

Matthew27 = [
       ['Matthew','Matt','27','001','1',"When the morning was come, all the chief priests and elders of the people took counsel against Jesus to put him to death:"],
['Matthew','Matt','27','002','2',"And when they had bound him, they led \emph{him} away, and delivered him to Pontius Pilate the governor."],
['Matthew','Matt','27','003','3',"Then Judas, which had betrayed him, when he saw that he was condemned, repented himself, and brought again the thirty pieces of silver to the chief priests and elders,"],
['Matthew','Matt','27','004','4',"Saying, I have sinned in that I have betrayed the innocent blood. And they said, What \emph{is} \emph{that} to us? see thou \emph{to} \emph{that}."],
['Matthew','Matt','27','005','5',"And he cast down the pieces of silver in the temple, and departed, and went and hanged himself."],
['Matthew','Matt','27','006','6',"And the chief priests took the silver pieces, and said, It is not lawful for to put them into the treasury, because it is the price of blood."],
['Matthew','Matt','27','007','7',"And they took counsel, and bought with them the potter's field, to bury strangers in."],
['Matthew','Matt','27','008','8',"Wherefore that field was called, The field of blood, unto this day."],
['Matthew','Matt','27','009','9',"Then was fulfilled that which was spoken by Jeremy the prophet, saying, And they took the thirty pieces of silver, the price of him that was valued, whom they of the children of Israel did value;"],
['Matthew','Matt','27','010','10',"And gave them for the potter's field, as the Lord appointed me."],
['Matthew','Matt','27','011','11',"And Jesus stood before the governor: and the governor asked him, saying, Art thou the King of the Jews? And Jesus said unto him, Thou sayest."],
['Matthew','Matt','27','012','12',"And when he was accused of the chief priests and elders, he answered nothing."],
['Matthew','Matt','27','013','13',"Then said Pilate unto him, Hearest thou not how many things they witness against thee?"],
['Matthew','Matt','27','014','14',"And he answered him to never a word; insomuch that the governor marvelled greatly."],
['Matthew','Matt','27','015','15',"Now at \emph{that} feast the governor was wont to release unto the people a prisoner, whom they would."],
['Matthew','Matt','27','016','16',"And they had then a notable prisoner, called Barabbas."],
['Matthew','Matt','27','017','17',"Therefore when they were gathered together, Pilate said unto them, Whom will ye that I release unto you? Barabbas, or Jesus which is called Christ?"],
['Matthew','Matt','27','018','18',"For he knew that for envy they had delivered him."],
['Matthew','Matt','27','019','19',"When he was set down on the judgment seat, his wife sent unto him, saying, Have thou nothing to do with that just man: for I have suffered many things this day in a dream because of him."],
['Matthew','Matt','27','020','20',"But the chief priests and elders persuaded the multitude that they should ask Barabbas, and destroy Jesus."],
['Matthew','Matt','27','021','21',"The governor answered and said unto them, Whether of the twain will ye that I release unto you? They said, Barabbas."],
['Matthew','Matt','27','022','22',"Pilate saith unto them, What shall I do then with Jesus which is called Christ? \emph{They} all say unto him, Let him be crucified."],
['Matthew','Matt','27','023','23',"And the governor said, Why, what evil hath he done? But they cried out the more, saying, Let him be crucified."],
['Matthew','Matt','27','024','24',"When Pilate saw that he could prevail nothing, but \emph{that} rather a tumult was made, he took water, and washed \emph{his} hands before the multitude, saying, I am innocent of the blood of this just person: see ye \emph{to} \emph{it}."],
['Matthew','Matt','27','025','25',"Then answered all the people, and said, His blood \emph{be} on us, and on our children."],
['Matthew','Matt','27','026','26',"Then released he Barabbas unto them: and when he had scourged Jesus, he delivered \emph{him} to be crucified."],
['Matthew','Matt','27','027','27',"Then the soldiers of the governor took Jesus into the common hall, and gathered unto him the whole band \emph{of} \emph{soldiers}."],
['Matthew','Matt','27','028','28',"And they stripped him, and put on him a scarlet robe."],
['Matthew','Matt','27','029','29',"And when they had platted a crown of thorns, they put \emph{it} upon his head, and a reed in his right hand: and they bowed the knee before him, and mocked him, saying, Hail, King of the Jews!"],
['Matthew','Matt','27','030','30',"And they spit upon him, and took the reed, and smote him on the head."],
['Matthew','Matt','27','031','31',"And after that they had mocked him, they took the robe off from him, and put his own raiment on him, and led him away to crucify \emph{him}."],
['Matthew','Matt','27','032','32',"And as they came out, they found a man of Cyrene, Simon by name: him they compelled to bear his cross."],
['Matthew','Matt','27','033','33',"And when they were come unto a place called Golgotha, that is to say, a place of a skull,"],
['Matthew','Matt','27','034','34',"They gave him vinegar to drink mingled with gall: and when he had tasted \emph{thereof}, he would not drink."],
['Matthew','Matt','27','035','35',"And they crucified him, and parted his garments, casting lots: that it might be fulfilled which was spoken by the prophet, They parted my garments among them, and upon my vesture did they cast lots."],
['Matthew','Matt','27','036','36',"And sitting down they watched him there;"],
['Matthew','Matt','27','037','37',"And set up over his head his accusation written, THIS IS JESUS THE KING OF THE JEWS."],
['Matthew','Matt','27','038','38',"Then were there two thieves crucified with him, one on the right hand, and another on the left."],
['Matthew','Matt','27','039','39',"And they that passed by reviled him, wagging their heads,"],
['Matthew','Matt','27','040','40',"And saying, Thou that destroyest the temple, and buildest \emph{it} in three days, save thyself. If thou be the Son of God, come down from the cross."],
['Matthew','Matt','27','041','41',"Likewise also the chief priests mocking \emph{him}, with the scribes and elders, said,"],
['Matthew','Matt','27','042','42',"He saved others; himself he cannot save. If he be the King of Israel, let him now come down from the cross, and we will believe him."],
['Matthew','Matt','27','043','43',"He trusted in God; let him deliver him now, if he will have him: for he said, I am the Son of God."],
['Matthew','Matt','27','044','44',"The thieves also, which were crucified with him, cast the same in his teeth."],
['Matthew','Matt','27','045','45',"Now from the sixth hour there was darkness over all the land unto the ninth hour."],
['Matthew','Matt','27','046','46',"And about the ninth hour Jesus cried with a loud voice, saying, Eli, Eli, lama sabachthani? that is to say, My God, my God, why hast thou forsaken me?"],
['Matthew','Matt','27','047','47',"Some of them that stood there, when they heard \emph{that}, said, This \emph{man} calleth for Elias."],
['Matthew','Matt','27','048','48',"And straightway one of them ran, and took a spunge, and filled \emph{it} with vinegar, and put \emph{it} on a reed, and gave him to drink."],
['Matthew','Matt','27','049','49',"The rest said, Let be, let us see whether Elias will come to save him."],
['Matthew','Matt','27','050','50',"Jesus, when he had cried again with a loud voice, yielded up the ghost."],
['Matthew','Matt','27','051','51',"And, behold, the veil of the temple was rent in twain from the top to the bottom; and the earth did quake, and the rocks rent;"],
['Matthew','Matt','27','052','52',"And the graves were opened; and many bodies of the saints which slept arose,"],
['Matthew','Matt','27','053','53',"And came out of the graves after his resurrection, and went into the holy city, and appeared unto many."],
['Matthew','Matt','27','054','54',"Now when the centurion, and they that were with him, watching Jesus, saw the earthquake, and those things that were done, they feared greatly, saying, Truly this was the Son of God."],
['Matthew','Matt','27','055','55',"And many women were there beholding afar off, which followed Jesus from Galilee, ministering unto him:"],
['Matthew','Matt','27','056','56',"Among which was Mary Magdalene, and Mary the mother of James and Joses, and the mother of Zebedee's children."],
['Matthew','Matt','27','057','57',"When the even was come, there came a rich man of Arimathaea, named Joseph, who also himself was Jesus' disciple:"],
['Matthew','Matt','27','058','58',"He went to Pilate, and begged the body of Jesus. Then Pilate commanded the body to be delivered."],
['Matthew','Matt','27','059','59',"And when Joseph had taken the body, he wrapped it in a clean linen cloth,"],
['Matthew','Matt','27','060','60',"And laid it in his own new tomb, which he had hewn out in the rock: and he rolled a great stone to the door of the sepulchre, and departed."],
['Matthew','Matt','27','061','61',"And there was Mary Magdalene, and the other Mary, sitting over against the sepulchre."],
['Matthew','Matt','27','062','62',"Now the next day, that followed the day of the preparation, the chief priests and Pharisees came together unto Pilate,"],
['Matthew','Matt','27','063','63',"Saying, Sir, we remember that that deceiver said, while he was yet alive, After three days I will rise again."],
['Matthew','Matt','27','064','64',"Command therefore that the sepulchre be made sure until the third day, lest his disciples come by night, and steal him away, and say unto the people, He is risen from the dead: so the last error shall be worse than the first."],
['Matthew','Matt','27','065','65',"Pilate said unto them, Ye have a watch: go your way, make \emph{it} as sure as ye can."],
['Matthew','Matt','27','066','66',"So they went, and made the sepulchre sure, sealing the stone, and setting a watch."]
]

Matthew28 = [
             ['Matthew','Matt','28','001','1',"In the end of the sabbath, as it began to dawn toward the first \emph{day} of the week, came Mary Magdalene and the other Mary to see the sepulchre."],
['Matthew','Matt','28','002','2',"And, behold, there was a great earthquake: for the angel of the Lord descended from heaven, and came and rolled back the stone from the door, and sat upon it."],
['Matthew','Matt','28','003','3',"His countenance was like lightning, and his raiment white as snow:"],
['Matthew','Matt','28','004','4',"And for fear of him the keepers did shake, and became as dead \emph{men}."],
['Matthew','Matt','28','005','5',"And the angel answered and said unto the women, Fear not ye: for I know that ye seek Jesus, which was crucified."],
['Matthew','Matt','28','006','6',"He is not here: for he is risen, as he said. Come, see the place where the Lord lay."],
['Matthew','Matt','28','007','7',"And go quickly, and tell his disciples that he is risen from the dead; and, behold, he goeth before you into Galilee; there shall ye see him: lo, I have told you."],
['Matthew','Matt','28','008','8',"And they departed quickly from the sepulchre with fear and great joy; and did run to bring his disciples word."],
['Matthew','Matt','28','009','9',"And as they went to tell his disciples, behold, Jesus met them, saying, All hail. And they came and held him by the feet, and worshipped him."],
['Matthew','Matt','28','010','10',"Then said Jesus unto them, Be not afraid: go tell my brethren that they go into Galilee, and there shall they see me."],
['Matthew','Matt','28','011','11',"Now when they were going, behold, some of the watch came into the city, and shewed unto the chief priests all the things that were done."],
['Matthew','Matt','28','012','12',"And when they were assembled with the elders, and had taken counsel, they gave large money unto the soldiers,"],
['Matthew','Matt','28','013','13',"Saying, Say ye, His disciples came by night, and stole him \emph{away} while we slept."],
['Matthew','Matt','28','014','14',"And if this come to the governor's ears, we will persuade him, and secure you."],
['Matthew','Matt','28','015','15',"So they took the money, and did as they were taught: and this saying is commonly reported among the Jews until this day."],
['Matthew','Matt','28','016','16',"Then the eleven disciples went away into Galilee, into a mountain where Jesus had appointed them."],
['Matthew','Matt','28','017','17',"And when they saw him, they worshipped him: but some doubted."],
['Matthew','Matt','28','018','18',"And Jesus came and spake unto them, saying, All power is given unto me in heaven and in earth."],
['Matthew','Matt','28','019','19',"Go ye therefore, and teach all nations, baptizing them in the name of the Father, and of the Son, and of the Holy Ghost:"],
['Matthew','Matt','28','020','20',"Teaching them to observe all things whatsoever I have commanded you: and, lo, I am with you alway, \emph{even} unto the end of the world. Amen."]
]

def main():
    ProcessVerses('Matthew16',Matthew16)
      
if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{The Pharisees also with the Sadducees came, and tempting desired him that he would shew them a sign from heaven.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{He answered and said unto them, When it is evening, ye say, \emph{It} \emph{will} \emph{be} fair weather: for the sky is red.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And in the morning, \emph{It} \emph{will} \emph{be} foul weather to day: for the sky is red and lowring. O \emph{ye} hypocrites, ye can discern the face of the sky; but can ye not \emph{discern} the signs of the times?}
[4] \textcolor[cmyk]{0.99998,1,0,0}{A wicked and adulterous generation seeketh after a sign; and there shall no sign be given unto it, but the sign of the prophet Jonas. And he left them, and departed.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{And when his d

\emph{man} & 1 \\ \hline
\emph{say} & 1 \\ \hline
\emph{that} & 1 \\ \hline
\emph{them} & 1 \\ \hline
\emph{thou} & 1 \\ \hline
\emph{will} & 2 \\ \hline
\emph{ye} & 1 \\ \hline
a & 4 \\ \hline
according & 1 \\ \hline
adulterous & 1 \\ \hline
after & 2 \\ \hline
again & 1 \\ \hline
against & 1 \\ \hline
also & 2 \\ \hline
am & 2 \\ \hline
among & 2 \\ \hline
an & 1 \\ \hline
and & 32 \\ \hline
angels & 1 \\ \hline
answered & 3 \\ \hline
any & 1 \\ \hline
art & 4 \\ \hline
asked & 1 \\ \hline
bade & 1 \\ \hline
baskets & 2 \\ \hline
be & 9 \\ \hline
because & 2 \\ \hline
began & 2 \\ \hline
behind & 1 \\ \hline
beware & 3 \\ \hline
bind & 1 \\ \hline
blood & 1 \\ \hline
bound & 1 \\ \hline
bread & 5 \\ \hline
brought & 1 \\ \hline
build & 1 \\ \hline
but & 5 \\ \hline
came & 2 \\ \hline
can & 2 \\ \hline
charged & 1 \\ \hline
chief & 1 \\ \hline
church & 1 \\ \hline
coasts & 1 \\ \hline
come & 3 \\ \hline
coming & 1 \\ \hline
concerning & 1 \\ \hline
cross & 1 \\ \hline
day & 2 \\ \hlin